# 导包+数据简单处理

In [1]:
pip install category_encoders

     |████████████████████████████████| 82 kB 147 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install optuna

     |████████████████████████████████| 362 kB 1.5 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xgboost

     |████████████████████████████████| 255.9 MB 73 kB/s              █████▍                         | 50.7 MB 753 kB/s eta 0:04:33
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import re
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectFromModel, RFE
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
train = pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_train.csv')
predict = pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_test.csv')
fullData = pd.concat([train,predict],axis=0)
xiaoqu = pd.read_csv("/home/mw/project/xiaoqu_rent.csv")

In [6]:
is_matching = fullData['小区名称'].isin(xiaoqu['名称'])

# 获取匹配的索引
matching_indexes = fullData.index[is_matching]

not_matching = ~fullData['小区名称'].isin(xiaoqu['名称'])
# 获取不匹配的索引
unmatching_indexes = fullData.index[not_matching]

# 获取不匹配的小区名称
unmatched_names = fullData.loc[not_matching, '小区名称']

In [7]:
#按照最近的经纬度匹配小区
def haversine(lon1, lat1, lon2, lat2):
    """
    计算两个经纬度点之间的距离（单位：千米）
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

    # 对于匹配的索引的行，直接添加匹配的小区名称
fullData.loc[matching_indexes, '匹配小区'] = fullData.loc[matching_indexes, '小区名称']

# 对于不匹配的索引的行，根据经纬度匹配最近的 xiaoqu 的名称
for index, row in fullData.loc[unmatching_indexes].iterrows():
    distances = haversine(row['lon'], row['lat'], xiaoqu['coord_x'], xiaoqu['coord_y'])
    nearest_index = np.argmin(distances)
    fullData.loc[index, '匹配小区'] = xiaoqu.loc[nearest_index, '名称']

In [8]:
columns_to_merge = ['建筑年代','房屋总数','楼栋总数','绿 化 率', '容 积 率','物 业 费','停车位','停车费用','环线位置','平均每平米月租金']
fullData = fullData.merge(xiaoqu[['名称'] + columns_to_merge],
                          left_on='匹配小区', right_on='名称', how='left')

#补充环线信息
fullData['环线'] = fullData['环线'].combine_first(fullData['环线'])
fullData['环线'].fillna("无", inplace=True)

# 删除多余的“名称”列
fullData = fullData.drop(columns=['名称', '环线位置'])
fullData.head()

,城市,区域,板块,环线,小区名称,价格,房屋户型,所在楼层,建筑面积,套内面积,...,匹配小区,建筑年代,房屋总数,楼栋总数,绿 化 率,容 积 率,物 业 费,停车位,停车费用,平均每平米月租金
0,0,79.0,111.0,二至三环,人定湖西里,6564200.0,2室1厅1厨1卫,中楼层 (共5层),52.3㎡,NaN,...,人定湖西里,1982.0,1317,19,30.0,3.00,1.3-1.65元/月/㎡,300.0,0.0,143.621275
1,0,43.0,231.0,五至六环,龙跃苑四区,4174000.0,3室1厅1厨1卫,顶层 (共6层),127.44㎡,123.7㎡,...,龙跃苑四区,2005.0,2317,40,30.0,1.73,0.65元/月/㎡,1550.0,150.0,65.334173
2,0,97.0,54.0,五至六环,名都园,16310000.0,4室2厅1厨4卫,底层 (共3层),228.54㎡,NaN,...,名都园,2002.0,1249,565,30.1,0.64,2-10元/月/㎡,0.0,0.0,123.502865
3,0,62.0,568.0,三至四环,保利海德公园,2834600.0,2房间2卫,低楼层 (共10层),43.6㎡,29.39㎡,...,保利海德公园,2015.0,577,12,40.0,2.60,6.18-20元/月/㎡,950.0,1150.0,234.292104
4,0,62.0,226.0,三至四环,京投银泰琨御府,1954000.0,1房间1卫,中楼层 (共10层),39.85㎡,29.94㎡,...,京投银泰琨御府,2010.0,1685,19,60.0,1.58,4.8-5.5元/月/㎡,1800.0,1200.0,172.492516


In [9]:
df = fullData.loc[:84132]
predict_data = fullData.loc[84133:]

In [10]:
print("合并后行数:", len(predict_data))
train_df, test_df = train_test_split(df, test_size=0.15, random_state=111)

合并后行数: 14786


In [11]:
xiaoqu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2973 entries, 0 to 2972
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   名称        2973 non-null   object 
 1   环线位置      1730 non-null   object 
 2   建筑年代      2793 non-null   float64
 3   房屋总数      2973 non-null   int64  
 4   楼栋总数      2973 non-null   int64  
 5   物业公司      2344 non-null   object 
 6   绿 化 率     2973 non-null   float64
 7   容 积 率     2973 non-null   float64
 8   物 业 费     2630 non-null   object 
 9   供水        2798 non-null   object 
 10  供暖        2273 non-null   object 
 11  供电        2812 non-null   object 
 12  燃气费       2564 non-null   object 
 13  供热费       1190 non-null   object 
 14  停车位       2973 non-null   float64
 15  停车费用      2973 non-null   float64
 16  coord_x   2973 non-null   float64
 17  coord_y   2973 non-null   float64
 18  匹配租房      2973 non-null   object 
 19  小区名称      2973 non-null   object 
 20  平均每平米月租金  2973 non-null   floa

In [12]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, PolynomialFeatures


def preprocess_full(df, is_training=True, encoders=None, transformers=None, use_poly=True):
    """
    统一处理分类变量、数值变量、文本提取等特征工程操作。
    
    参数:
        df: 待处理的DataFrame
        is_training: 是否为训练阶段
        encoders: 分类变量的编码器（如 OneHotEncoder）
        transformers: 数值变量的变换器（如 imputer、scaler、poly）
        use_poly: 是否对数值变量做多项式扩展

    返回:
        - 训练集: df, encoders, transformers
        - 测试集: df
    """
    df = df.copy()

    # 1. 删除重复与无效列
    df = df.drop_duplicates().dropna(axis=1, how='all')
    drop_cols = ['交易权属','年份', '环线位置', '小区地址', '物业类别',
                 '上次交易', '房屋用途', '房屋年限', '产权所属', '套内面积', '别墅类型']
    df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)

    # 2. 时间处理
    if '交易时间' in df.columns:
        df['交易时间'] = pd.to_datetime(df['交易时间'], errors='coerce')
        df['交易年份'] = df['交易时间'].dt.year
        df.drop(columns=['交易时间'], inplace=True)

    # 3. 填充字段 & 编码处理
    if '装修情况' in df.columns:
        df['装修情况'] = df['装修情况'].fillna('其他')
        df['装修编码'] = df['装修情况'].map({'简装': '简装', '精装': '精装', '毛坯': '毛坯'}).fillna('其他')
    else:
        df['装修编码'] = '其他'

    if '配备电梯' in df.columns:
        df['配备电梯'] = df['配备电梯'].fillna('无').map({'有': 1, '无': 0})
    else:
        df['配备电梯'] = 0

    if '建筑结构' in df.columns:
        df['建筑结构'] = df['建筑结构'].fillna('钢混结构')
        df['建筑结构编码'] = df['建筑结构'].map({'混合结构': '混合结构', '钢混结构': '钢混结构'}).fillna('其他')
    else:
        df['建筑结构编码'] = '其他'

    # 4. OneHot 编码
    onehot_cols = ['建筑结构编码', '装修编码']
    existing_cols = [col for col in onehot_cols if col in df.columns]

    if is_training:
        encoders = {} if encoders is None else encoders
        encoder = OneHotEncoder(handle_unknown="ignore", drop="first", sparse=False)
        encoders['onehot_encoder'] = encoder

        if existing_cols:
            onehot_encoded = encoder.fit_transform(df[existing_cols])
            col_names = encoder.get_feature_names_out(existing_cols)
            onehot_df = pd.DataFrame(onehot_encoded, columns=col_names, index=df.index)
            df.drop(columns=existing_cols, inplace=True)
            df = pd.concat([df, onehot_df], axis=1)
    else:
        if existing_cols and encoders and 'onehot_encoder' in encoders:
            encoder = encoders['onehot_encoder']
            onehot_encoded = encoder.transform(df[existing_cols])
            col_names = encoder.get_feature_names_out(existing_cols)
            onehot_df = pd.DataFrame(onehot_encoded, columns=col_names, index=df.index)
            df.drop(columns=existing_cols, inplace=True)
            df = pd.concat([df, onehot_df], axis=1)

    # 5. 文本字段数值提取
    def safe_extract_digits(x):
        if pd.isna(x):
            return np.nan
        matches = re.findall(r'\d+\.?\d*', str(x))
        return float(matches[0]) if matches else np.nan

    numeric_text_cols = {
        '房屋总数': '房屋数量',
        '楼栋总数': '楼栋数量',
        '绿 化 率': '绿化率',
        '容 积 率': '容积率',
        '物 业 费': '物业费',
        '停车位': '停车位数量',
        '停车费用': '停车费'
    }

    for col, new_col in numeric_text_cols.items():
        if col in df.columns:
            df[new_col] = df[col].apply(safe_extract_digits)

    numerical_cols = ['房屋数量', '楼栋数量', '绿化率', '容积率', '物业费', '停车位数量', '停车费']
    X_num = df[numerical_cols].copy()

    # 6. 数值变量处理
    # 6.1 缺失值填补
    if is_training:
        imputer = SimpleImputer(strategy='mean')
        X_num_imputed = imputer.fit_transform(X_num)
        transformers = transformers or {}
        transformers['imputer'] = imputer
    else:
        imputer = transformers['imputer']
        X_num_imputed = imputer.transform(X_num)

    # 6.2 裁剪离群值
    def clip_outliers(X):
        lower = np.percentile(X, 5, axis=0)
        upper = np.percentile(X, 95, axis=0)
        return np.clip(X, lower, upper)

    X_clipped = clip_outliers(X_num_imputed)

    # 6.3 log1p 变换
    X_log = np.log1p(np.clip(X_clipped, a_min=0, a_max=None))

    # 6.4 标准化
    if is_training:
        scaler = RobustScaler()
        X_scaled = scaler.fit_transform(X_log)
        transformers['scaler'] = scaler
    else:
        scaler = transformers['scaler']
        X_scaled = scaler.transform(X_log)

    # 6.5 多项式扩展（可选）
    if use_poly:
        if is_training:
            poly = PolynomialFeatures(degree=2, include_bias=False)
            X_poly = poly.fit_transform(X_scaled)
            transformers['poly'] = poly
            col_names = [f'poly_{name}' for name in poly.get_feature_names_out(numerical_cols)]
        else:
            poly = transformers['poly']
            X_poly = poly.transform(X_scaled)
            col_names = [f'poly_{name}' for name in poly.get_feature_names_out(numerical_cols)]

        X_num_df = pd.DataFrame(X_poly, columns=col_names, index=df.index)
    else:
        X_num_df = pd.DataFrame(X_scaled, columns=numerical_cols, index=df.index)

    # 删除冲突列并拼接
    df.drop(columns=df.columns.intersection(X_num_df.columns), inplace=True)
    df = pd.concat([df, X_num_df], axis=1)

    # 最后删除原始分类字段
    df.drop(columns=['建筑结构', '装修情况'], errors='ignore', inplace=True)

    if is_training:
        return df, encoders, transformers
    else:
        return df

In [13]:
train_df, encoders,transformers = preprocess_full(train_df, is_training=True)
test_df = preprocess_full(test_df, is_training=False, encoders=encoders, transformers=transformers)
predict_data = preprocess_full(predict_data, is_training=False, encoders=encoders, transformers=transformers)

# 建筑面积

In [14]:
def clean_area_column(df):
    """清洗‘建筑面积’列的非数字字符并转换为浮点数"""
    df['建筑面积'] = (
        df['建筑面积']
        .astype(str)
        .str.replace(r'[^0-9.]', '', regex=True)
        .astype(float)
    )
    return df

# 分别应用到每个数据集
train_df = clean_area_column(train_df)
test_df = clean_area_column(test_df)
predict_data = clean_area_column(predict_data)

# Step 2. 数据验证
# 检查转换结果
print("清洗后示例：\n", train_df['建筑面积'].sample(5))
print("\n异常值统计：")
print("零或负面积:", train_df[train_df['建筑面积'] <=0].shape[0])
print("缺失值比例:", train_df['建筑面积'].isnull().mean())
print("建筑面积描述统计：\n", train_df['建筑面积'].describe())

清洗后示例：
 16211    130.65
36069     89.46
50959     95.58
16767     62.44
11874     57.24
Name: 建筑面积, dtype: float64

异常值统计：
零或负面积: 0
缺失值比例: 0.0
建筑面积描述统计：
 count    71458.000000
mean        96.641668
std         65.316834
min         10.000000
25%         66.000000
50%         88.870000
75%        115.960000
max      10337.000000
Name: 建筑面积, dtype: float64


In [15]:
#计算99%分位数作为截断点
area_99_percentile = train_df['建筑面积'].quantile(0.99)

#截断异常值
train_df.loc[train_df['建筑面积'] > area_99_percentile, '建筑面积'] = area_99_percentile
test_df.loc[test_df['建筑面积'] > area_99_percentile, '建筑面积'] = area_99_percentile
predict_data.loc[predict_data['建筑面积'] > area_99_percentile, '建筑面积'] = area_99_percentile

# 创建管道：先标准化，再添加多项式特征
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False))
])
    
# 仅选择建筑面积列
area_train = train_df[["建筑面积"]]
area_test = test_df[["建筑面积"]]
area_predict=predict_data[["建筑面积"]]
    
# 转换训练集
area_poly_train = pipeline.fit_transform(area_train)
# 使用训练集的均值和标准差转换测试集
area_poly_test = pipeline.transform(area_test)
area_poly_predict = pipeline.transform(area_predict)
    
# 将转换后的特征添加回原始数据框
train_df = train_df.copy()
train_df["area_scaled"] = area_poly_train[:, 0]
train_df["area_squared"] = area_poly_train[:, 1]
    
test_df = test_df.copy()
test_df["area_scaled"] = area_poly_test[:, 0]
test_df["area_squared"] = area_poly_test[:, 1]
    
predict_data = predict_data.copy()
predict_data["area_scaled"] = area_poly_predict[:, 0]
predict_data["area_squared"] = area_poly_predict[:, 1]

In [16]:
# 检查数据集
print("当前所有列名:", train_df.columns.tolist())

当前所有列名: ['城市', '区域', '板块', '环线', '小区名称', '价格', '房屋户型', '所在楼层', '建筑面积', '房屋朝向', '梯户比例', '配备电梯', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '匹配小区', '建筑年代', '房屋总数', '楼栋总数', '绿 化 率', '容 积 率', '物 业 费', '停车位', '停车费用', '平均每平米月租金', '交易年份', '建筑结构编码_混合结构', '建筑结构编码_钢混结构', '装修编码_毛坯', '装修编码_简装', '装修编码_精装', '房屋数量', '楼栋数量', '绿化率', '容积率', '物业费', '停车位数量', '停车费', 'poly_房屋数量', 'poly_楼栋数量', 'poly_绿化率', 'poly_容积率', 'poly_物业费', 'poly_停车位数量', 'poly_停车费', 'poly_房屋数量^2', 'poly_房屋数量 楼栋数量', 'poly_房屋数量 绿化率', 'poly_房屋数量 容积率', 'poly_房屋数量 物业费', 'poly_房屋数量 停车位数量', 'poly_房屋数量 停车费', 'poly_楼栋数量^2', 'poly_楼栋数量 绿化率', 'poly_楼栋数量 容积率', 'poly_楼栋数量 物业费', 'poly_楼栋数量 停车位数量', 'poly_楼栋数量 停车费', 'poly_绿化率^2', 'poly_绿化率 容积率', 'poly_绿化率 物业费', 'poly_绿化率 停车位数量', 'poly_绿化率 停车费', 'poly_容积率^2', 'poly_容积率 物业费', 'poly_容积率 停车位数量', 'poly_容积率 停车费', 'poly_物业费^2', 'poly_物业费 停车位数量', 'poly_物业费 停车费', 'poly_停车位数量^2', 'poly_停车位数量 停车费', 'poly_停车费^2', 'area_scaled', 'area_squared']


# 房屋朝向

In [17]:
#房屋朝向处理
def extract_main_direction(orientation):
    """
    规则优先级：南 > 北 > 东 > 西
    参数：
        orientation: 字符串（如"东南北"）或NaN
    返回：
        主要朝向（"南","北","东","西","其他"）
    """
    if pd.isna(orientation):
        return "其他"
    orientation = str(orientation).replace(" ", "")  # 清洗空格
    
    # 按优先级判断
    if "南" in orientation:
        return "南"
    elif "北" in orientation:
        return "北"
    elif "东" in orientation:
        return "东"
    elif "西" in orientation:
        return "西"
    else:
        return "其他"

# 训练集处理
train_df["主要朝向"] = train_df["房屋朝向"].apply(extract_main_direction)

# 验证集处理（必须使用相同处理方式）
test_df["主要朝向"] = test_df["房屋朝向"].apply(extract_main_direction)

# 预测集处理
predict_data["主要朝向"] = predict_data["房屋朝向"].apply(extract_main_direction)
        
# 初始化编码器（自动处理未知类别）
encoder = OneHotEncoder(handle_unknown="ignore", drop="first", sparse=False)
encoder.fit(train_df[["主要朝向"]])  # 只在训练集上fit

# 训练集
train_encoded = encoder.transform(train_df[["主要朝向"]])
train_df = pd.concat([
    train_df.drop(["房屋朝向", "主要朝向"], axis=1),
    pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(["主要朝向"]), index=train_df.index)
], axis=1)

# 验证集
test_encoded = encoder.transform(test_df[["主要朝向"]])
test_df = pd.concat([
    test_df.drop(["房屋朝向", "主要朝向"], axis=1),
    pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(["主要朝向"]), index=test_df.index)
], axis=1)

# 预测集
pred_encoded = encoder.transform(predict_data[["主要朝向"]])
predict_data = pd.concat([
    predict_data.drop(["房屋朝向", "主要朝向"], axis=1),
    pd.DataFrame(pred_encoded, columns=encoder.get_feature_names_out(["主要朝向"]), index=predict_data.index)
], axis=1)

# 城市_环线

In [18]:
print(train_df['环线'].isnull().sum())  # 缺失数量
print(train_df.groupby('城市')['环线'].apply(lambda x: x.isnull().mean()))  # 每个城市的缺失比例
city_region_ring_counts = train_df.groupby(['城市', '板块'])['环线'].nunique().reset_index()
city_region_ring_counts.rename(columns={'环线': '环线类别数'}, inplace=True)
# 筛选环线类别数 > 2 的 (城市, 板块)
city_region_unstable = city_region_ring_counts[city_region_ring_counts['环线类别数'] > 2]

# 显示结果
print(city_region_unstable)

# 统计这些 (城市, 板块) 组合的个数
num_unstable_regions = city_region_unstable.shape[0]
print(f"环线类别数大于 2 的 (城市, 板块) 组合个数: {num_unstable_regions}")

0
城市
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
Name: 环线, dtype: float64
     城市     板块  环线类别数
0     0    1.0      3
56    0  226.0      3
177   0  686.0      3
188   0  749.0      3
229   2   70.0      3
235   2  122.0      3
254   2  246.0      3
257   2  269.0      3
268   2  376.0      3
323   2  804.0      3
325   2  809.0      3
329   3   18.0      3
337   3   56.0      3
354   3  152.0      3
466   3  699.0      3
476   3  732.0      3
490   4    6.0      3
492   4   22.0      3
495   4   50.0      3
497   4   74.0      3
498   4  102.0      3
508   4  192.0      3
509   4  195.0      3
518   4  273.0      3
520   4  314.0      3
525   4  366.0      4
532   4  462.0      3
535   4  466.0      3
539   4  476.0      3
541   4  486.0      3
542   4  489.0      3
543   4  496.0      3
544   4  538.0      3
546   4  555.0      3
556   4  649.0      3
557   4  700.0      3
559   4  704.0      3
562   4  726.0      3
563   4  738.0      3
564   4  741.0      3
566  

In [19]:
def train_fill_missing_环线(train_df):
    """仅在训练集计算填补规则"""
   
    # 规则1: 城市1和5的环线固定为"无环线"
    city_special_rules = {1 : '无环线', 5 : '无环线'}
   
    # 规则2: 按板块→区域→城市层级计算环线众数
    板块_环线_map = train_df.groupby("板块")["环线"].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan).to_dict()
    区域_环线_map = train_df.groupby("区域")["环线"].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan).to_dict()
    城市_环线_map = train_df.groupby("城市")["环线"].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan).to_dict()
   
    return {
        'city_special_rules': city_special_rules,
        '板块_环线_map': 板块_环线_map,
        '区域_环线_map': 区域_环线_map,
        '城市_环线_map': 城市_环线_map
    }

def apply_fill_missing_环线(df, fill_rules):
    """应用填补规则到任意数据集"""
   
    # 深拷贝避免修改原数据
    df = df.copy()
   
    # 应用特殊城市规则
    special_cities = fill_rules['city_special_rules'].keys()
    df.loc[df["城市"].astype(int).isin(special_cities), "环线"] = df["城市"].map(fill_rules['city_special_rules'])
   
    # 按层级填补
    df["环线_填补"] = (
        df["环线"]
        .fillna(df["板块"].map(fill_rules['板块_环线_map']))
        .fillna(df["区域"].map(fill_rules['区域_环线_map']))
        .fillna(df["城市"].map(fill_rules['城市_环线_map']))
    )
   
    return df

# 只在训练集计算填补规则
fill_rules = train_fill_missing_环线(train_df)

# 应用规则到所有数据集
train_df = apply_fill_missing_环线(train_df, fill_rules)
test_df = apply_fill_missing_环线(test_df, fill_rules)
predict_data = apply_fill_missing_环线(predict_data, fill_rules)

# 创建城市_环线变量
for df in [train_df, test_df, predict_data]:
    df["城市_环线"] = df["城市"].astype(str) + "_" + df["环线_填补"].astype(str)

In [20]:
#绘图-城市_环线与建筑面积
plt.figure(figsize=(12, 8))
sns.lmplot(
    x='建筑面积',
    y='价格',
    hue='城市_环线',
    data=train_df,
    height=6,
    aspect=1.5,
    scatter_kws={'alpha': 0.3},  # 设置点透明度
    line_kws={'linewidth': 2}    # 设置回归线粗细
)
plt.title('建筑面积 vs 价格（按城市环线分组）', fontsize=14)
plt.xlabel('建筑面积 (㎡)', fontsize=12)
plt.ylabel('价格', fontsize=12)
plt.show()

<Figure size 864x576 with 0 Axes>

<Figure size 745.125x432 with 1 Axes>

In [21]:
# 初始化编码器（参数不变）
encoder = OneHotEncoder(
    sparse=True,
    handle_unknown="ignore",
    drop="first"
)

# 只在训练集拟合
encoder.fit(train_df[["城市_环线"]])

# 转换所有数据集
def encode_city_ring(df, encoder):
    encoded = encoder.transform(df[["城市_环线"]])
    
    # 修改为兼容新版pandas的代码
    try:
        # 尝试使用老方法
        encoded_df = pd.DataFrame.sparse.from_spmatrix(
            encoded,
            columns=encoder.get_feature_names_out(["城市_环线"]),
            index=df.index
        )
    except AttributeError:
        # 如果不支持，使用新方法
        encoded_df = pd.DataFrame(
            encoded.toarray(),
            columns=encoder.get_feature_names_out(["城市_环线"]),
            index=df.index
        )
    return pd.concat([df.drop("城市_环线", axis=1), encoded_df], axis=1)

train_df = encode_city_ring(train_df, encoder)
test_df = encode_city_ring(test_df, encoder)
predict_data = encode_city_ring(predict_data, encoder)

# 检查填补后缺失值
for name, df in zip(["Train", "Test", "Predict"], [train_df, test_df, predict_data]):
    print(f"{name}环线缺失数:", df["环线_填补"].isna().sum())

# 查看编码特征维度（应全部相同）
print("编码特征:", encoder.get_feature_names_out())

Train环线缺失数: 0
Test环线缺失数: 0
Predict环线缺失数: 0
编码特征: ['城市_环线_0_二环内' '城市_环线_0_二至三环' '城市_环线_0_五至六环' '城市_环线_0_六环外' '城市_环线_0_四至五环'
 '城市_环线_0_无' '城市_环线_1_无环线' '城市_环线_2_内环内' '城市_环线_2_内环至外环' '城市_环线_2_外环外'
 '城市_环线_2_无' '城市_环线_3_中环至外环' '城市_环线_3_内环内' '城市_环线_3_内环至中环' '城市_环线_3_外环外'
 '城市_环线_3_无' '城市_环线_4_一环内' '城市_环线_4_一至二环' '城市_环线_4_三至四环' '城市_环线_4_二至三环'
 '城市_环线_4_四环外' '城市_环线_4_无' '城市_环线_5_无环线' '城市_环线_6_一环内' '城市_环线_6_一至二环'
 '城市_环线_6_三环外' '城市_环线_6_二至三环' '城市_环线_6_无']


In [22]:
#创建交互项
city_ring_columns = [col for col in train_df.columns if col.startswith("城市_环线_")]
def add_interaction_terms(df, city_ring_cols):
    for col in city_ring_cols:
        df[f"{col}_建筑面积"] = df[col] * df["建筑面积"]
    return df

train_df = add_interaction_terms(train_df, city_ring_columns)
test_df = add_interaction_terms(test_df, city_ring_columns)
predict_data = add_interaction_terms(predict_data, city_ring_columns)

from sklearn.preprocessing import StandardScaler
import numpy as np

# 仅标准化交互项
interaction_columns = [col for col in train_df.columns if "城市_环线_" in col and "建筑面积" in col]

# 为每个交互项分别处理
for col in interaction_columns:
    # 检查是否是稀疏数据并适当处理
    scaler = StandardScaler(with_mean=False)
    
    # 训练集
    if hasattr(train_df[col], 'sparse'):
        # 如果是SparseArray类型
        train_df[col] = scaler.fit_transform(np.array(train_df[col]).reshape(-1, 1))
    elif isinstance(train_df[col].array, pd.arrays.SparseArray):
        # 新版pandas中的稀疏数组处理
        train_df[col] = scaler.fit_transform(train_df[col].array.to_numpy().reshape(-1, 1))
    else:
        # 常规数组处理
        train_df[col] = scaler.fit_transform(np.array(train_df[col]).reshape(-1, 1))
    
    # 测试集 (同样的处理逻辑)
    if hasattr(test_df[col], 'sparse'):
        test_df[col] = scaler.transform(np.array(test_df[col]).reshape(-1, 1))
    elif isinstance(test_df[col].array, pd.arrays.SparseArray):
        test_df[col] = scaler.transform(test_df[col].array.to_numpy().reshape(-1, 1))
    else:
        test_df[col] = scaler.transform(np.array(test_df[col]).reshape(-1, 1))
    
    # 预测集 (同样的处理逻辑)
    if hasattr(predict_data[col], 'sparse'):
        predict_data[col] = scaler.transform(np.array(predict_data[col]).reshape(-1, 1))
    elif isinstance(predict_data[col].array, pd.arrays.SparseArray):
        predict_data[col] = scaler.transform(predict_data[col].array.to_numpy().reshape(-1, 1))
    else:
        predict_data[col] = scaler.transform(np.array(predict_data[col]).reshape(-1, 1))

# 梯户比例

In [23]:
def process_elevator_household_ratio(df):
    """处理梯户比例字段"""
    # 填充缺失值（用默认值 "1梯2户" 代替 "无"）
    df['梯户比例'] = df['梯户比例'].fillna('1梯2户')
    
    def extract_ratio(ratio_str):
        if not isinstance(ratio_str, str):
            return np.nan
        
        # 更灵活的正则匹配（支持 "2梯4户", "2部电梯4户", "2T4H" 等）
        match = re.search(r'(\d+)\s*[梯部T]\s*(\d+)\s*户?', ratio_str)
        if match:
            elevators = int(match.group(1))
            households = int(match.group(2))
            return households / elevators if elevators > 0 else np.nan
        return np.nan  # 如果无法解析，返回 NaN（后续会填充）
    
    # 计算户梯比
    df['户梯比'] = df['梯户比例'].apply(extract_ratio)
    
    # 先按小区填充，再用全局中位数填充剩余的缺失值
    global_median = df['户梯比'].median()
    df['户梯比'] = df.groupby('小区名称')['户梯比'].transform(
        lambda x: x.fillna(x.median() if not x.isnull().all() else global_median)
    )
    df['户梯比'] = df['户梯比'].fillna(global_median)
    
    return df

# 处理数据
train_df = process_elevator_household_ratio(train_df)
test_df = process_elevator_household_ratio(test_df)
predict_data = process_elevator_household_ratio(predict_data)

In [24]:
print("处理后 train_df 的缺失值统计:")
print(train_df['户梯比'].isna().sum())

print("\n处理后 test_df 的缺失值统计:")
print(test_df['户梯比'].isna().sum())

print("\n处理后 predict_data 的缺失值统计:")
print(predict_data['户梯比'].isna().sum())

处理后 train_df 的缺失值统计:
0

处理后 test_df 的缺失值统计:
0

处理后 predict_data 的缺失值统计:
0


# 厅室数量

In [25]:
# 处理房屋户型（室厅厨卫）
def extract_room_info(house_type):
    """提取房屋户型信息的函数"""
    # 检查输入是否为字符串，非字符串的情况返回默认值
    if not isinstance(house_type, str):
        return {'室': 0, '厅': 0, '厨': 0, '卫': 0}
    
    # 使用正则表达式提取室、厅、厨、卫的数据
    room_data = re.findall(r'(\d+)室|(\d+)厅|(\d+)厨|(\d+)卫', house_type)
    
    # 默认初始值为0
    room_count = {'室': 0, '厅': 0, '厨': 0, '卫': 0}
    
    # 提取匹配的数据
    for match in room_data:
        for i, category in enumerate(['室', '厅', '厨', '卫']):
            if match[i]:
                room_count[category] = int(match[i])
    
    return room_count

def process_house_type(train_df, test_df, predict_data):
    """处理所有数据集的房屋户型"""
    # 1. 填充缺失值为空字符串，避免正则表达式报错
    for df in [train_df, test_df, predict_data]:
        df['房屋户型'] = df['房屋户型'].fillna('')
    
    # 2. 提取户型信息到新列
    for df in [train_df, test_df, predict_data]:
        for room_type in ['室', '厅', '厨', '卫']:
            df[room_type] = df['房屋户型'].apply(lambda x: extract_room_info(x)[room_type])
    
    # 3. 计算训练集的中位数，用于填充缺失值
    room_medians = {
        '室': train_df['室'].median(),
        '厅': train_df['厅'].median()
    }
    
    # 4. 使用中位数和常见值填充缺失值
    for df in [train_df, test_df, predict_data]:
        df['室'] = df['室'].fillna(room_medians['室'])
        df['厅'] = df['厅'].fillna(room_medians['厅'])
        df['厨'] = df['厨'].fillna(1)  # 使用常见值1填充
        df['卫'] = df['卫'].fillna(1)  # 使用常见值1填充
    
    return train_df, test_df, predict_data

# 处理所有数据集的房屋户型
train_df, test_df, predict_data = process_house_type(train_df, test_df, predict_data)

#  区域、板块

In [26]:
train_df['log_价格'] = np.log1p(train_df['价格'])
test_df['log_价格'] = np.log1p(test_df['价格'])
# 对区域、板块进行 Target Encoding，区域板块类别多，使用独热编码维度过高
#训练集：fit_transform，需要使用目标变量 log_价格
target_enc = TargetEncoder(cols=['区域', '板块'])
train_df[['区域', '板块']] = target_enc.fit_transform(train_df[['区域', '板块']], train_df['log_价格'])
#验证集
test_df[['区域', '板块']] = target_enc.transform(test_df[['区域', '板块']])
#测试集：直接 transform（测试集没有价格信息）
predict_data[['区域', '板块']] = target_enc.transform(predict_data[['区域', '板块']])

#对 Target Encoding 后的区域和板块进行归一化到 [-1,1]，有利于后续正则化
minmax_scaler = MinMaxScaler(feature_range=(-1, 1))
train_df[['区域', '板块']] = minmax_scaler.fit_transform(train_df[['区域', '板块']])
test_df[['区域', '板块']] = minmax_scaler.transform(test_df[['区域', '板块']])
predict_data[['区域', '板块']] = minmax_scaler.transform(predict_data[['区域', '板块']])

# 自然语言处理

## 高频词

In [27]:
from collections import Counter

In [28]:
#因为核心卖点与户型介绍有很多重复的部分，所以合并起来
#train_df['卖点-户型'] = train_df['核心卖点'] + train_df['户型介绍']
#test_df['卖点-户型'] = test_df['核心卖点'] + test_df['户型介绍']
#predict_data['卖点-户型'] = predict_data['核心卖点'] + predict_data['户型介绍']

In [29]:
#寻找高频词函数
def extract_high_frequency_words(df, column_name, top_n=10):
    # 将指定列的空值替换为空字符串
    df[column_name] = df[column_name].fillna('')

    # 定义一个函数用于分词
    def tokenize(text):
        # 使用正则表达式匹配中文词语
        words = re.findall(r'[\u4e00-\u9fa5]+', text)
        return words

    # 对指定列进行分词
    all_words = []
    for description in df[column_name]:
        words = tokenize(description)
        all_words.extend(words)

    # 统计词频
    word_counts = Counter(all_words)

    # 显示前 top_n 个高频词
    return word_counts.most_common(top_n)

In [30]:
print("周边配套高频词:",extract_high_frequency_words(train_df,'周边配套', top_n=33))
print("交通出行高频词:",extract_high_frequency_words(train_df,'交通出行', top_n=20))
print("房屋优势高频词:",extract_high_frequency_words(train_df,'房屋优势', top_n=20))

周边配套高频词: [('米', 8489), ('医院', 5517), ('市', 4514), ('公里', 3612), ('建设银行', 3414), ('银行', 3400), ('工商银行', 3011), ('商场', 2737), ('路', 2591), ('公园', 2530), ('永辉', 2294), ('中国银行', 2189), ('农业银行', 2032), ('号线', 2011), ('华润万家', 1361), ('家乐福', 1353), ('沃尔玛', 1315), ('地铁', 1273), ('医疗', 1165), ('购物', 1165), ('配套齐全', 1018), ('交通银行', 1003), ('人人乐', 944), ('电影院', 904), ('小区', 898), ('出行方便', 881), ('生活便利', 838), ('物美', 824), ('距离', 809), ('交通便利', 776), ('距离小区', 764), ('万达广场', 738), ('招商银行', 737)]
交通出行高频词: [('路', 76280), ('米', 12439), ('号线', 11583), ('地铁', 5562), ('公里', 4802), ('出行方便', 4236), ('公交', 3278), ('有', 3270), ('交通便利', 3165), ('轻轨', 2003), ('等', 1937), ('距离', 1726), ('路等', 1494), ('出行便利', 1490), ('米左右', 1352), ('公交线路', 1286), ('线', 1217), ('路公交车', 1177), ('距离地铁', 1078), ('公交站', 1000)]
房屋优势高频词: [('装修', 29647), ('地铁', 26470), ('房本满五年', 25600), ('房本满两年', 13882)]


In [31]:
pip install jieba

     |████████████████████████████████| 19.2 MB 814 kB/s            07
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314476 sha256=bb49b6a353346199075088afc67be8a3c502a80fccf13b77bb6d6e7f5dcb16e3
  Stored in directory: /home/mw/.cache/pip/wheels/24/aa/17/5bc7c72e9a37990a9620cc3aad0acad1564dcff6dbc2359de3
Successfully built jieba
Note: you may need to restart the kernel to use updated packages.


In [32]:
import re
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import jieba
import numpy as np

class RealEstateTextProcessor:
    def __init__(self):
        # 优化后的关键词字典 - 增加权重和同义词
        self.keyword_categories = {
            "交通": {
                "keywords": ["地铁", "公交", "轻轨", "主干道", "站点", "交通", "号线", "路", "米", "BRT", "快速路", "高架"],
                "weight": 1.2,  # 交通权重稍高
                "synonyms": {"地铁": ["轨道交通", "城铁"], "公交": ["巴士", "班车"]}
            },
            "配套": {
                "keywords": ["医院", "诊所", "超市", "商场", "商圈", "购物", "百货", "广场", "公园", "银行", 
                           "图书馆", "绿地", "电影院", "家乐福", "永辉", "沃尔玛", "华润万家", "物美", "人人乐"],
                "weight": 1.0,
                "synonyms": {"医院": ["三甲", "医疗"], "超市": ["便利店"]}
            },
            "学区": {
                "keywords": ["学校", "学区", "重点", "幼儿园", "小学", "中学", "大学", "名校", "示范"],
                "weight": 1.3,  # 学区房权重最高
                "synonyms": {"重点": ["名校", "示范"], "学区": ["教育"]}
            },
            "税费": {
                "keywords": ["满五唯一", "税费", "免税", "满两年", "满五年", "个税", "契税"],
                "weight": 0.8,
                "synonyms": {}
            }
        }
        
        # 初始化分词器
        jieba.initialize()
        
    def smart_tokenize(self, text):
        """智能分词 - 结合正则和jieba"""
        if pd.isna(text) or text == '':
            return []
        
        # 先用jieba分词
        jieba_words = list(jieba.cut(text))
        # 再用正则提取中文词语
        regex_words = re.findall(r'[\u4e00-\u9fa5]+', text)
        
        # 合并并去重
        all_words = list(set(jieba_words + regex_words))
        return [w for w in all_words if len(w) > 1]  # 过滤单字
    
    def extract_enhanced_keywords(self, text, category_info):
        """增强版关键词提取 - 支持同义词和权重"""
        if pd.isna(text):
            return 0
        
        keywords = category_info["keywords"]
        synonyms = category_info["synonyms"]
        weight = category_info["weight"]
        
        score = 0
        words = self.smart_tokenize(text)
        
        for word in words:
            # 直接匹配
            if word in keywords:
                score += weight
            # 同义词匹配
            for key, syn_list in synonyms.items():
                if word in syn_list:
                    score += weight * 0.8  # 同义词权重稍低
        
        return min(score, 3)  # 限制最大得分，避免过度偏重
    
    def calculate_text_diversity(self, df, text_columns):
        """计算文本多样性得分"""
        def diversity_score(row):
            all_text = ' '.join([str(row[col]) for col in text_columns if pd.notna(row[col])])
            words = self.smart_tokenize(all_text)
            if len(words) == 0:
                return 0
            # 计算词汇丰富度
            unique_words = len(set(words))
            total_words = len(words)
            return unique_words / total_words if total_words > 0 else 0
        
        return df.apply(diversity_score, axis=1)
    
    def extract_selling_points_tfidf(self, df, text_col='核心卖点', top_n=5):
        """使用TF-IDF提取核心卖点特征"""
        # 清理文本
        texts = df[text_col].fillna('').apply(lambda x: ' '.join(self.smart_tokenize(x)))
        
        if texts.str.len().sum() == 0:  # 如果没有文本内容
            return pd.DataFrame()
        
        # TF-IDF向量化
        vectorizer = TfidfVectorizer(
            max_features=50,
            ngram_range=(1, 2),  # 支持1-2元词组
            min_df=2,
            token_pattern=r'(?u)\b\w+\b'
        )
        
        try:
            tfidf_matrix = vectorizer.fit_transform(texts)
            feature_names = vectorizer.get_feature_names_out()
            
            # 为每个房源计算TF-IDF得分最高的特征
            tfidf_scores = tfidf_matrix.toarray()
            top_features = []
            
            for scores in tfidf_scores:
                top_indices = scores.argsort()[-top_n:][::-1]
                top_words = [feature_names[i] for i in top_indices if scores[i] > 0]
                top_features.append('_'.join(top_words[:3]))  # 取前3个作为特征
            
            return pd.Series(top_features, index=df.index)
        except:
            return pd.Series([''] * len(df), index=df.index)
    
    def process_all_features(self, df, text_columns):
        """处理所有文本特征"""
        df_processed = df.copy()
        
        # 1. 增强版关键词特征
        for category, info in self.keyword_categories.items():
            df_processed[f"{category}_得分"] = 0
            for col in text_columns:
                if col in df_processed.columns:
                    scores = df_processed[col].apply(
                        lambda x: self.extract_enhanced_keywords(x, info)
                    )
                    df_processed[f"{category}_得分"] += scores
        
        # 2. 文本多样性特征
        df_processed["描述丰富度"] = self.calculate_text_diversity(df_processed, text_columns)
        
        # 3. TF-IDF核心卖点特征
        if '核心卖点' in df_processed.columns:
            df_processed["核心卖点_特征"] = self.extract_selling_points_tfidf(df_processed)
        
        # 4. 文本长度特征
        for col in text_columns:
            if col in df_processed.columns:
                df_processed[f"{col}_长度"] = df_processed[col].fillna('').str.len()
        
        # 5. 综合评分
        score_cols = [f"{cat}_得分" for cat in self.keyword_categories.keys()]
        df_processed["综合文本得分"] = df_processed[score_cols].sum(axis=1) + df_processed["描述丰富度"] * 2
        
        return df_processed

# 使用示例
def process_datasets(train_df, test_df, predict_data):
    """处理所有数据集"""
    processor = RealEstateTextProcessor()
    text_columns = ["房屋优势", "核心卖点", "周边配套", "交通出行"]
    
    # 处理所有数据集
    train_df = processor.process_all_features(train_df, text_columns)
    test_df = processor.process_all_features(test_df, text_columns)
    predict_data = processor.process_all_features(predict_data, text_columns)
    
    # 按小区填补缺失的得分特征
    score_columns = [f"{cat}_得分" for cat in processor.keyword_categories.keys()]
    
    for df in [train_df, test_df, predict_data]:
        for col in score_columns:
            df[col] = df.groupby("小区名称")[col].transform(
                lambda x: x.fillna(x.mean()) if x.mean() > 0 else x.fillna(0)
            )
    
    return train_df, test_df, predict_data

# 调用处理函数
train_df, test_df, predict_data = process_datasets(train_df, test_df, predict_data)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.840 seconds.
Prefix dict has been built successfully.


In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
import warnings
warnings.filterwarnings('ignore')

def preprocess_for_xgboost(train_df, test_df, predict_data):
    """
    为XGBoost优化特征预处理
    """
    # 复制数据避免修改原始数据
    train_processed = train_df.copy()
    test_processed = test_df.copy() 
    predict_processed = predict_data.copy()
    
    # 1. 文本长度特征 - Log变换处理极端值
    length_cols = [col for col in train_processed.columns if col.endswith('_长度')]
    
    for col in length_cols:
        for df in [train_processed, test_processed, predict_processed]:
            # Log(x+1)变换，避免log(0)
            df[f"{col}_log"] = np.log1p(df[col])
            # 可选：保留原特征或删除
            # df.drop(col, axis=1, inplace=True)
    
    # 2. 综合文本得分 - RobustScaler（基于训练集拟合）
    if '综合文本得分' in train_processed.columns:
        scaler = RobustScaler()
        
        # 只用训练集拟合
        train_score = train_processed[['综合文本得分']].values
        scaler.fit(train_score)
        
        # 应用到所有数据集
        train_processed['综合文本得分_scaled'] = scaler.transform(train_score).flatten()
        test_processed['综合文本得分_scaled'] = scaler.transform(
            test_processed[['综合文本得分']].values).flatten()
        predict_processed['综合文本得分_scaled'] = scaler.transform(
            predict_processed[['综合文本得分']].values).flatten()
    
    # 3. 描述丰富度 - 通常不需要处理，但可以创建分箱特征
    if '描述丰富度' in train_processed.columns:
        # 创建分箱特征（可选）
        bins = [0, 0.3, 0.6, 0.8, 1.0]
        labels = ['低', '中低', '中高', '高']
        
        for df in [train_processed, test_processed, predict_processed]:
            df['描述丰富度_分档'] = pd.cut(df['描述丰富度'], bins=bins, labels=labels, include_lowest=True)
            # 转换为数值型
            df['描述丰富度_分档'] = df['描述丰富度_分档'].cat.codes
    
    # 4. 处理核心卖点特征（字符串转数值）
    if '核心卖点_特征' in train_processed.columns:
        # 收集所有唯一值
        all_unique = set()
        for df in [train_processed, test_processed, predict_processed]:
            all_unique.update(df['核心卖点_特征'].unique())
        
        # 创建映射字典
        feature_map = {feature: idx for idx, feature in enumerate(sorted(all_unique))}
        
        # 应用映射
        for df in [train_processed, test_processed, predict_processed]:
            df['核心卖点_特征_encoded'] = df['核心卖点_特征'].map(feature_map).fillna(-1)
    
    # 5. 创建特征交互（XGBoost会自动学习，但手动创建几个有用的）
    interaction_features = []
    
    for df in [train_processed, test_processed, predict_processed]:
        # 学区 × 交通（学区房的交通便利性）
        if '学区_得分' in df.columns and '交通_得分' in df.columns:
            df['学区交通_交互'] = df['学区_得分'] * df['交通_得分']
            interaction_features.append('学区交通_交互')
        
        # 配套 × 描述丰富度（配套好且描述详细）
        if '配套_得分' in df.columns and '描述丰富度' in df.columns:
            df['配套描述_交互'] = df['配套_得分'] * df['描述丰富度']
            interaction_features.append('配套描述_交互')
    
    print("✅ XGBoost预处理完成！")
    print(f"📊 新增特征: {len(length_cols)} 个Log变换 + 综合得分标准化 + 特征交互")
    
    return train_processed, test_processed, predict_processed

# 6. XGBoost特征重要性分析辅助函数
def analyze_text_features(model, feature_names):
    """
    分析文本特征在XGBoost中的重要性
    """
    if hasattr(model, 'feature_importances_'):
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        # 筛选文本相关特征
        text_features = importance_df[
            importance_df['feature'].str.contains('得分|长度|丰富度|卖点|交互')
        ].head(10)
        
        print("🔥 文本特征重要性 TOP10:")
        for _, row in text_features.iterrows():
            print(f"   {row['feature']}: {row['importance']:.4f}")
        
        return text_features
    
    return None

# 使用示例
train_df, test_df, predict_data = preprocess_for_xgboost(train_df, test_df, predict_data)

✅ XGBoost预处理完成！
📊 新增特征: 4 个Log变换 + 综合得分标准化 + 特征交互


In [34]:
print("当前所有列名:", train_df.columns.tolist())

当前所有列名: ['城市', '区域', '板块', '环线', '小区名称', '价格', '房屋户型', '所在楼层', '建筑面积', '梯户比例', '配备电梯', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '匹配小区', '建筑年代', '房屋总数', '楼栋总数', '绿 化 率', '容 积 率', '物 业 费', '停车位', '停车费用', '平均每平米月租金', '交易年份', '建筑结构编码_混合结构', '建筑结构编码_钢混结构', '装修编码_毛坯', '装修编码_简装', '装修编码_精装', '房屋数量', '楼栋数量', '绿化率', '容积率', '物业费', '停车位数量', '停车费', 'poly_房屋数量', 'poly_楼栋数量', 'poly_绿化率', 'poly_容积率', 'poly_物业费', 'poly_停车位数量', 'poly_停车费', 'poly_房屋数量^2', 'poly_房屋数量 楼栋数量', 'poly_房屋数量 绿化率', 'poly_房屋数量 容积率', 'poly_房屋数量 物业费', 'poly_房屋数量 停车位数量', 'poly_房屋数量 停车费', 'poly_楼栋数量^2', 'poly_楼栋数量 绿化率', 'poly_楼栋数量 容积率', 'poly_楼栋数量 物业费', 'poly_楼栋数量 停车位数量', 'poly_楼栋数量 停车费', 'poly_绿化率^2', 'poly_绿化率 容积率', 'poly_绿化率 物业费', 'poly_绿化率 停车位数量', 'poly_绿化率 停车费', 'poly_容积率^2', 'poly_容积率 物业费', 'poly_容积率 停车位数量', 'poly_容积率 停车费', 'poly_物业费^2', 'poly_物业费 停车位数量', 'poly_物业费 停车费', 'poly_停车位数量^2', 'poly_停车位数量 停车费', 'poly_停车费^2', 'area_scaled', 'area_squared', '主要朝向_北', '主要朝向_南', '主要朝向_西', '环线_填补', '城市_环线_0_二环内', '城市_环线_0_二至三环'

In [35]:
# 关键词字典
keywords = {
    "交通": ["地铁", "公交", "轻轨","主干道","站点","交通","号线","路","米"],
    "配套": ["医院", "诊所","诊所","超市", "商场", "商圈","购物", "百货", "广场", "公园", "银行", "图书馆","绿地","电影院",
             "家乐福","永辉","沃尔玛","华润万家","物美","人人乐"],
    "学区": ["学校", "学区","重点","幼儿园","小学","中学","大学"],
    "税费": ["满五唯一","税费","免税","满两年","满五年"],
}

# 处理文本特征的函数
def extract_keywords(text, keyword_list):
    if pd.isna(text):
        return 0  # 缺失值填充为0
    for word in keyword_list:
        if word in text:
            return 1  # 只要出现关键词，就记为1
    return 0

# 需要处理的字段
text_columns = ["房屋优势", "核心卖点", "周边配套", "交通出行"]

#处理所有数据集
def process_text_features(df):
    for category, words in keywords.items():
        df[f"{category}_关键词"] = 0
        for col in text_columns:
            df[f"{category}_关键词"] |= df[col].apply(lambda x: extract_keywords(x, words))
    # 处理户型介绍变量
    df["户型介绍_是否填写"] = df["户型介绍"].notna().astype(int)
    return df

# 处理所有数据集
train_df = process_text_features(train_df)
test_df = process_text_features(test_df)
predict_data = process_text_features(predict_data)

# 按小区填补缺失值（针对“交通_关键词”和“配套_关键词”）
def fill_missing_by_group(df, group_col, fill_cols):
    for col in fill_cols:
        df[col] = df.groupby(group_col)[col].transform(lambda x: x.fillna(x.max()))
    return df

fill_columns = ["交通_关键词", "配套_关键词","学区_关键词"]

# 对所有数据集按小区填补
train_df = fill_missing_by_group(train_df, "小区名称", fill_columns)
test_df = fill_missing_by_group(test_df, "小区名称", fill_columns)
predict_data = fill_missing_by_group(predict_data, "小区名称", fill_columns)

# 计算匹配的关键词类别数量
def count_keyword_categories(df):
    df["关键词匹配数量"] = (
        df["交通_关键词"] + 
        df["配套_关键词"] + 
        df["税费_关键词"] +
        df["户型介绍_是否填写"]
    )
    return df

# 处理所有数据集
train_df = count_keyword_categories(train_df)
test_df = count_keyword_categories(test_df)
predict_data = count_keyword_categories(predict_data)

In [36]:
print("train_df 类型:", type(train_df))

train_df 类型: <class 'pandas.core.frame.DataFrame'>


# 楼层处理

In [37]:
def extract_floor_info(s):
    try:
        
        # 优先检查是否包含地下室
        if "地下室" in s:
            # 地下室逻辑：总楼层设为1，当前楼层设为0或-1（根据业务需求）
            return -1, 1  # 当前楼层=0（表示地下室），总楼层=1
        total = re.search(r'共(\d+)层', s).group(1)
        level = re.search(r'(\d+)层', s.split('(')[0])

        if level:
            current = level.group(1)
        else:
            desc = s.split('(')[0].strip()
            total = int(total)
            if '低' in desc:
                current = total * 0.25
            elif '中' in desc:
                current = total * 0.5
            elif '高' in desc:
                current = total * 0.75
            elif '底' in desc:
                current = 1
            else:
                current = total * 1
        return int(float(current)), int(total)
    except:
        return np.nan, np.nan

for df in [train_df,test_df,predict_data]:
    df[['当前楼层', '总楼层']] = df['所在楼层'].apply(
        lambda x: pd.Series(extract_floor_info(x) if isinstance(x, str) else (np.nan, np.nan)))

# 检查 数据集 里是否有 '总楼层'
print(train_df.columns)  
print(predict_data.columns)

def add_floor_features(*dfs):
    """
    批量添加楼层相关特征到多个DataFrame（直接修改原DataFrame）
    参数:
        *dfs: 一个或多个包含"总楼层"和"当前楼层"的DataFrame
    """
    for df in dfs:
        # 1. 基本特征
        df["多层住宅"] = (df["总楼层"] <= 6).astype(int)
        
        # 2. 黄金楼层特征
        df["黄金楼层"] = (
            ((df["多层住宅"] == 1) & (df["当前楼层"].between(2, 4))) |
            ((df["多层住宅"] == 0) & (df["当前楼层"] >= (df["总楼层"] * 0.5)))
        ).astype(int)
        
        # 3. 楼层占比特征
        df["楼层占比"] = df["当前楼层"] / df["总楼层"]
        df["楼层占比_平方"] = df["楼层占比"] ** 2
        
        # 4. 交互特征
        df["楼层_电梯交互"] = df["楼层占比"] * df["多层住宅"]
        df["楼层_高层交互"] = df["楼层占比"] * (1 - df["多层住宅"])

# 处理所有数据集
add_floor_features(train_df, test_df, predict_data)

Index(['城市', '区域', '板块', '环线', '小区名称', '价格', '房屋户型', '所在楼层', '建筑面积', '梯户比例',
       ...
       '学区交通_交互', '配套描述_交互', '交通_关键词', '配套_关键词', '学区_关键词', '税费_关键词',
       '户型介绍_是否填写', '关键词匹配数量', '当前楼层', '总楼层'],
      dtype='object', length=172)
Index(['城市', '区域', '板块', '环线', '小区名称', '房屋户型', '所在楼层', '建筑面积', '梯户比例', '配备电梯',
       ...
       '学区交通_交互', '配套描述_交互', '交通_关键词', '配套_关键词', '学区_关键词', '税费_关键词',
       '户型介绍_是否填写', '关键词匹配数量', '当前楼层', '总楼层'],
      dtype='object', length=171)


In [38]:
def extract_floor_info(s):
    try:
        if not isinstance(s, str):
            return np.nan, np.nan
            
        # 处理地下室情况
        if "地下室" in s:
            return -1, 1
            
        # 匹配总楼层：从括号内提取"共XX层"
        total_match = re.search(r'共(\d+)层', s)
        if not total_match:
            return np.nan, np.nan
        total = int(total_match.group(1))
        
        # 匹配当前楼层描述（括号前部分）
        desc = s.split('(')[0].strip()
        
        # 处理明确楼层数字的情况（如"3层 (共26层)"）
        level_match = re.search(r'(\d+)层', desc)
        if level_match:
            return int(level_match.group(1)), total
            
        # 处理文字描述（低/中/高/底层）
        if '低' in desc:
            if total <= 2:  # 总楼层为1或2时，当前楼层取1
                current = 1
            else:
                current = round(total * 0.25)
        elif '中' in desc:
            current = round(total * 0.5)
        elif '高' in desc:
            current = round(total * 0.75)
        elif '底' in desc:
            current = 1
        else:  # 顶层
            current = total
            
        return min(int(current), total), total  # 确保不超过总楼层
    except:
        return np.nan, np.nan

for df in [train_df,test_df,predict_data]:
    df[['当前楼层', '总楼层']] = df['所在楼层'].apply(
        lambda x: pd.Series(extract_floor_info(x) if isinstance(x, str) else (np.nan, np.nan)))

In [39]:
# 检查训练集
print("训练集中当前楼层为0的记录数量:", train_df[train_df["当前楼层"] == 0].shape[0])

nan_count = train_df['当前楼层'].isna().sum()
print("当前楼层中NaN值的个数:", nan_count)

训练集中当前楼层为0的记录数量: 0
当前楼层中NaN值的个数: 0


In [40]:
def add_floor_features(*dfs):
    """
    批量添加楼层相关特征到多个DataFrame（直接修改原DataFrame）
    参数:
        *dfs: 一个或多个包含"总楼层"和"当前楼层"的DataFrame
    """
    for df in dfs:
        # 1. 基本特征
        df["多层住宅"] = (df["总楼层"] <= 6).astype(int)
        
        # 2. 黄金楼层特征
        df["黄金楼层"] = (
            ((df["多层住宅"] == 1) & (df["当前楼层"].between(2, 4))) |
            ((df["多层住宅"] == 0) & (df["当前楼层"] >= (df["总楼层"] * 0.5)))
        ).astype(int)
        
        # 3. 楼层占比特征
        df["楼层占比"] = df["当前楼层"] / df["总楼层"]
        df["楼层占比_平方"] = df["楼层占比"] ** 2
        
        # 4. 交互特征
        df["楼层_电梯交互"] = df["楼层占比"] * df["多层住宅"]
        df["楼层_高层交互"] = df["楼层占比"] * (1 - df["多层住宅"])

# 使用示例：一次性处理所有数据集
add_floor_features(train_df, test_df, predict_data)

In [41]:
# 检查训练集
print("训练集中楼层占比为0的记录数量:", train_df[train_df["楼层占比"] == 0].shape[0])

nan_count = train_df['楼层占比'].isna().sum()
print("楼层占比中NaN值的个数:", nan_count)

训练集中楼层占比为0的记录数量: 0
楼层占比中NaN值的个数: 0


# 房龄处理

In [42]:
for df in [train_df, test_df, predict_data]:
    df['房龄'] = df['交易年份'] - df['建筑年代']

# 检查房龄范围
print(train_df['房龄'].describe())

count    68450.000000
mean        17.092403
std         10.290071
min          1.000000
25%          9.000000
50%         14.000000
75%         23.000000
max         69.000000
Name: 房龄, dtype: float64


In [43]:
print(train_df.groupby("多层住宅")["房龄"].describe())
# 计算相关性矩阵
corr_matrix = train_df[['房龄','多层住宅','建筑面积',"总楼层","容 积 率"]].corr()
print(corr_matrix['房龄'])  # 观察房龄与面积、总楼层的相关性
import seaborn as sns
# 绘制分组箱线图
import matplotlib.pyplot as plt
# 设置图形风格
sns.set_style("whitegrid")
plt.figure(figsize=(8, 6))

# 绘制箱线图
ax = sns.boxplot(x="多层住宅", y="房龄", data=train_df, palette="Set2")

# 添加标题和坐标轴标签
ax.set_title("Boxplot of Building Age by Multi-Story Residential", fontsize=14, fontweight='bold')
ax.set_xlabel("Multi-Story Residential", fontsize=12)
ax.set_ylabel("Building Age", fontsize=12)

# 调整刻度字体大小
ax.tick_params(axis='both', labelsize=10)

# 显示图形
plt.show()
# 检查缺失比例
missing_rate_by_group = train_df.groupby("多层住宅")["房龄"].apply(lambda x: x.isna().mean())
print(missing_rate_by_group)

        count       mean        std  min   25%   50%   75%   max
多层住宅                                                            
0     55513.0  15.462811   9.257333  1.0   9.0  13.0  20.0  54.0
1     12937.0  24.085027  11.498987  1.0  15.0  23.0  33.0  69.0
房龄       1.000000
多层住宅     0.328053
建筑面积    -0.176634
总楼层     -0.414447
容 积 率   -0.129483
Name: 房龄, dtype: float64


<Figure size 576x432 with 1 Axes>

多层住宅
0    0.045020
1    0.029337
Name: 房龄, dtype: float64


In [44]:
#1. 在训练集上计算各组的房龄中位数
median_age_multilayer = train_df[train_df["多层住宅"] == 1]["房龄"].median()  # 多层住宅（1）的中位数
median_age_non_multilayer = train_df[train_df["多层住宅"] == 0]["房龄"].median()  # 非多层住宅（0）的中位数

# 2. 定义填充函数
def fill_age_by_building_type(df, median_multilayer, median_non_multilayer):
    # 处理极端值
    df.loc[df["房龄"] > 100, "房龄"] = 100
    df.loc[df["房龄"] < 0, "房龄"] = np.nan
    
    # 分组填充缺失值
    df["房龄"] = df.apply(
        lambda row: median_multilayer if (pd.isna(row["房龄"]) and row["多层住宅"] == 1)
                   else (median_non_multilayer if (pd.isna(row["房龄"]) and row["多层住宅"] == 0)
                   else row["房龄"]),
        axis=1
    )
    return df

# 3. 应用填充
train_df = fill_age_by_building_type(train_df, median_age_multilayer, median_age_non_multilayer)
test_df = fill_age_by_building_type(test_df, median_age_multilayer, median_age_non_multilayer)
predict_data = fill_age_by_building_type(predict_data, median_age_multilayer, median_age_non_multilayer)

 #检查房龄列中的NaN数量
nan_count = train_df['房龄'].isna().sum()
print("房龄中NaN值的个数:", nan_count)

房龄中NaN值的个数: 0


In [45]:
train_df['面积_房龄'] = train_df['area_scaled'] * train_df['房龄']
test_df['面积_房龄'] = test_df['area_scaled'] * test_df['房龄']
predict_data['面积_房龄']=predict_data['area_scaled'] * predict_data['房龄']

# 对房龄做非线性转换
train_df['房龄_平方'] = train_df['房龄'] ** 2
test_df['房龄_平方'] = test_df['房龄'] ** 2
predict_data['房龄_平方'] = predict_data['房龄'] ** 2

# 对新特征进行标准化
new_features = ['面积_房龄', '房龄_平方']
scaler_new = StandardScaler()
train_df[new_features] = scaler_new.fit_transform(train_df[new_features])
test_df[new_features] = scaler_new.transform(test_df[new_features])
predict_data[new_features] = scaler_new.transform(predict_data[new_features])

# 需要标准化的特征
standard_features = ['房龄', '楼层占比','楼层占比_平方', '楼层_电梯交互', '楼层_高层交互']

# 对连续特征进行标准化
scaler = StandardScaler()
train_df[standard_features] = scaler.fit_transform(train_df[standard_features])
test_df[standard_features] = scaler.transform(test_df[standard_features])
predict_data[standard_features] = scaler.transform(predict_data[standard_features])

In [46]:
train_df['单位租金_平方'] = train_df['平均每平米月租金'] ** 2
test_df['单位租金_平方'] = test_df['平均每平米月租金'] ** 2
predict_data['单位租金_平方'] = predict_data['平均每平米月租金'] ** 2

train_df['卧室_租金'] = train_df['室'] * train_df['平均每平米月租金']
test_df['卧室_租金'] = test_df['室'] * test_df['平均每平米月租金']
predict_data['卧室_租金'] = predict_data['室'] * predict_data['平均每平米月租金']

train_df['面积_租金'] = train_df['area_scaled'] * train_df['平均每平米月租金']
test_df['面积_租金'] = test_df['area_scaled'] * test_df['平均每平米月租金']
predict_data['面积_租金'] = predict_data['area_scaled'] * predict_data['平均每平米月租金']

newnew_features = ['单位租金_平方', '卧室_租金','面积_租金']
scaler_new = StandardScaler()
train_df[newnew_features] = scaler_new.fit_transform(train_df[newnew_features])
test_df[newnew_features] = scaler_new.transform(test_df[newnew_features])
predict_data[newnew_features] = scaler_new.transform(predict_data[newnew_features])

In [47]:
# 检查数据集中的列
print("当前所有列名:", train_df.columns.tolist())

当前所有列名: ['城市', '区域', '板块', '环线', '小区名称', '价格', '房屋户型', '所在楼层', '建筑面积', '梯户比例', '配备电梯', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '匹配小区', '建筑年代', '房屋总数', '楼栋总数', '绿 化 率', '容 积 率', '物 业 费', '停车位', '停车费用', '平均每平米月租金', '交易年份', '建筑结构编码_混合结构', '建筑结构编码_钢混结构', '装修编码_毛坯', '装修编码_简装', '装修编码_精装', '房屋数量', '楼栋数量', '绿化率', '容积率', '物业费', '停车位数量', '停车费', 'poly_房屋数量', 'poly_楼栋数量', 'poly_绿化率', 'poly_容积率', 'poly_物业费', 'poly_停车位数量', 'poly_停车费', 'poly_房屋数量^2', 'poly_房屋数量 楼栋数量', 'poly_房屋数量 绿化率', 'poly_房屋数量 容积率', 'poly_房屋数量 物业费', 'poly_房屋数量 停车位数量', 'poly_房屋数量 停车费', 'poly_楼栋数量^2', 'poly_楼栋数量 绿化率', 'poly_楼栋数量 容积率', 'poly_楼栋数量 物业费', 'poly_楼栋数量 停车位数量', 'poly_楼栋数量 停车费', 'poly_绿化率^2', 'poly_绿化率 容积率', 'poly_绿化率 物业费', 'poly_绿化率 停车位数量', 'poly_绿化率 停车费', 'poly_容积率^2', 'poly_容积率 物业费', 'poly_容积率 停车位数量', 'poly_容积率 停车费', 'poly_物业费^2', 'poly_物业费 停车位数量', 'poly_物业费 停车费', 'poly_停车位数量^2', 'poly_停车位数量 停车费', 'poly_停车费^2', 'area_scaled', 'area_squared', '主要朝向_北', '主要朝向_南', '主要朝向_西', '环线_填补', '城市_环线_0_二环内', '城市_环线_0_二至三环'

In [48]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def find_optimal_k_clusters(data_scaled, k_range=None, max_k=15, min_samples_per_cluster=10,
                           early_stop=True, patience=3, min_improvement=0.02):
    """
    使用WSS（肘部法则）和轮廓系数来找到最优的聚类数K，支持早停机制
    
    参数:
    ◦ data_scaled: 标准化后的数据
    ◦ k_range: K值范围，如果为None则自动确定
    ◦ max_k: 最大K值
    ◦ min_samples_per_cluster: 每个簇的最少样本数
    ◦ early_stop: 是否启用早停机制
    ◦ patience: 早停耐心值（连续多少次没有改善就停止）
    ◦ min_improvement: 最小改善阈值
    
    返回:
    ◦ optimal_k: 最优K值
    ◦ wss_scores: WSS得分列表
    ◦ silhouette_scores: 轮廓系数列表
    ◦ k_values: 尝试的K值列表
    """
    
    n_samples = len(data_scaled)
    
    # 自动确定K值范围
    if k_range is None:
        min_k = 2
        # 确保每个簇至少有min_samples_per_cluster个样本
        max_k_by_samples = max(2, n_samples // min_samples_per_cluster)
        max_k = min(max_k, max_k_by_samples, 20)  # 限制最大值避免过度计算
        k_range = range(min_k, max_k + 1)
    
    print(f"寻找最优K值，范围: {list(k_range)}")
    if early_stop:
        print(f"早停设置: patience={patience}, min_improvement={min_improvement}")
    
    wss_scores = []
    silhouette_scores = []
    k_values = []
    
    # 早停相关变量
    best_silhouette = -1
    no_improvement_count = 0
    best_k_so_far = 2
    
    for k in k_range:
        if k >= n_samples:
            print(f"K={k} 超过样本数，停止搜索")
            break
            
        try:
            # 训练KMeans
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            cluster_labels = kmeans.fit_predict(data_scaled)
            
            # 计算WSS（簇内平方和）
            wss = kmeans.inertia_
            
            # 计算轮廓系数
            if k > 1:
                silhouette_avg = silhouette_score(data_scaled, cluster_labels)
            else:
                silhouette_avg = 0
            
            wss_scores.append(wss)
            silhouette_scores.append(silhouette_avg)
            k_values.append(k)
            
            print(f"K={k}: WSS={wss:.2f}, Silhouette={silhouette_avg:.3f}")
            
            # 早停逻辑
            if early_stop and k > 2:  # 至少尝试3个K值
                improvement = silhouette_avg - best_silhouette
                
                if improvement > min_improvement:
                    best_silhouette = silhouette_avg
                    best_k_so_far = k
                    no_improvement_count = 0
                    print(f"  ✓ 发现改善: +{improvement:.4f}")
                else:
                    no_improvement_count += 1
                    print(f"  → 无显著改善 ({no_improvement_count}/{patience})")
                    
                    # 检查是否需要早停
                    if no_improvement_count >= patience:
                        print(f"🛑 早停触发: 连续{patience}次无改善，当前最佳K={best_k_so_far}")
                        # 如果WSS还在快速下降，给一次机会
                        if len(wss_scores) >= 2:
                            wss_decline_rate = (wss_scores[-2] - wss_scores[-1]) / wss_scores[-2]
                            if wss_decline_rate > 0.1:  # WSS下降超过10%
                                print(f"  但WSS仍在快速下降({wss_decline_rate:.3f}), 继续搜索...")
                                no_improvement_count = patience - 1  # 重置但保持警戒
                            else:
                                break
            
        except Exception as e:
            print(f"K={k} 训练失败: {e}")
            continue
    
    if len(wss_scores) == 0:
        print("警告: 无法计算任何K值的聚类效果")
        return 2, [], [], []
    
    # 使用肘部法则找到最优K
    optimal_k = find_elbow_point(k_values, wss_scores)
    
    # 也可以考虑轮廓系数最高的K值作为参考
    if len(silhouette_scores) > 0:
        best_silhouette_k = k_values[np.argmax(silhouette_scores)]
        print(f"\n📊 结果分析:")
        print(f"肘部法则推荐K: {optimal_k}")
        print(f"轮廓系数最佳K: {best_silhouette_k}")
        if early_stop:
            print(f"早停建议K: {best_k_so_far}")
        
        # 综合决策逻辑
        candidates = [optimal_k, best_silhouette_k]
        if early_stop:
            candidates.append(best_k_so_far)
        
        # 选择出现频率最高的K，或者轮廓系数最高的
        from collections import Counter
        k_counter = Counter(candidates)
        most_common_k = k_counter.most_common(1)[0][0]
        
        if k_counter[most_common_k] > 1:
            optimal_k = most_common_k
            print(f"最终选择K: {optimal_k} (多方法一致)")
        else:
            optimal_k = best_silhouette_k
            print(f"最终选择K: {optimal_k} (优先轮廓系数)")
    
    return optimal_k, wss_scores, silhouette_scores, k_values

def find_elbow_point(k_values, wss_scores):
    """
    使用肘部法则找到最优K值
    计算每个点到连接首末两点直线的距离，距离最大的点为肘部
    """
    if len(wss_scores) <= 2:
        return k_values[0] if k_values else 2
    
    # 将数据转换为numpy数组
    k_array = np.array(k_values)
    wss_array = np.array(wss_scores)
    
    # 标准化数据到[0,1]范围
    k_norm = (k_array - k_array.min()) / (k_array.max() - k_array.min())
    wss_norm = (wss_array - wss_array.min()) / (wss_array.max() - wss_array.min())
    
    # 计算每个点到首末连线的距离
    first_point = np.array([k_norm[0], wss_norm[0]])
    last_point = np.array([k_norm[-1], wss_norm[-1]])
    
    distances = []
    for i in range(len(k_norm)):
        point = np.array([k_norm[i], wss_norm[i]])
        distance = point_to_line_distance(point, first_point, last_point)
        distances.append(distance)
    
    # 找到距离最大的点
    elbow_index = np.argmax(distances)
    optimal_k = k_values[elbow_index]
    
    return optimal_k

def point_to_line_distance(point, line_start, line_end):
    """计算点到直线的距离"""
    line_vec = line_end - line_start
    point_vec = point - line_start
    line_len = np.linalg.norm(line_vec)
    
    if line_len == 0:
        return np.linalg.norm(point_vec)
    
    line_unitvec = line_vec / line_len
    proj_length = np.dot(point_vec, line_unitvec)
    proj = proj_length * line_unitvec
    
    return np.linalg.norm(point_vec - proj)

def plot_clustering_metrics(k_values, wss_scores, silhouette_scores, optimal_k):
    """绘制聚类评估指标图"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # WSS肘部图
    ax1.plot(k_values, wss_scores, 'bo-')
    ax1.axvline(x=optimal_k, color='red', linestyle='--', label=f'Optimal K={optimal_k}')
    ax1.set_xlabel('Number of Clusters (K)')
    ax1.set_ylabel('Within-cluster Sum of Squares (WSS)')
    ax1.set_title('Elbow Method for Optimal K')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 轮廓系数图
    if len(silhouette_scores) > 0:
        ax2.plot(k_values, silhouette_scores, 'go-')
        ax2.axvline(x=optimal_k, color='red', linestyle='--', label=f'Selected K={optimal_k}')
        ax2.set_xlabel('Number of Clusters (K)')
        ax2.set_ylabel('Silhouette Score')
        ax2.set_title('Silhouette Score for Different K')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def compute_enhanced_location_features_improved(train_df, test_df=None, predict_data=None, 
                                              min_k=5, max_k=15, lightweight_mode=True,
                                              auto_select_clusters=True, plot_metrics=False,
                                              early_stop=True, patience=3):
    """
    改进版地理位置特征工程，使用WSS自动选择最优聚类数，支持早停
    
    新增参数:
    ◦ auto_select_clusters: 是否自动选择聚类数
    ◦ plot_metrics: 是否绘制聚类评估图
    ◦ early_stop: 是否启用早停机制
    ◦ patience: 早停耐心值
    """
    
    print("开始改进版地理位置特征工程...")
    
    # 检查必要列
    required_columns = ['价格', '建筑面积', 'lon', 'lat']
    for col in required_columns:
        if col not in train_df.columns:
            raise ValueError(f"训练集中未找到 '{col}' 列")
    
    # 处理训练集
    train_df = train_df.copy()
    train_df["单价"] = train_df["价格"] / train_df["建筑面积"]
    train_locations = train_df[['lon', 'lat']].values
    
    print(f"训练集样本数: {len(train_df)}")
    
    # 标准化地理位置坐标
    print("标准化地理位置坐标...")
    scaler = StandardScaler()
    train_locations_scaled = scaler.fit_transform(train_locations)
    
    # 计算数据中心
    data_center = np.mean(train_locations, axis=0)
    print(f"数据中心坐标: {data_center}")
    
    # 构建KNN模型
    print("构建KNN模型...")
    k_values = [5, 10, 15, 20] if not lightweight_mode else [5, 10]
    knn_models = {}
    
    for k in k_values:
        if k < len(train_df):
            knn_models[k] = NearestNeighbors(n_neighbors=k, metric='euclidean')
            knn_models[k].fit(train_locations_scaled)
            print(f"✓ KNN-{k} 模型训练完成")
    
    # 聚类分析
    cluster_model = None
    optimal_k = None
    
    if auto_select_clusters:
        print("自动选择最优聚类数...")
        optimal_k, wss_scores, silhouette_scores, k_values_tried = find_optimal_k_clusters(
            train_locations_scaled, max_k=max_k, early_stop=early_stop, patience=patience
        )
        
        if plot_metrics and len(wss_scores) > 0:
            plot_clustering_metrics(k_values_tried, wss_scores, silhouette_scores, optimal_k)
        
        # 使用最优K训练聚类模型
        if optimal_k and optimal_k < len(train_df):
            cluster_model = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
            cluster_model.fit(train_locations_scaled)
            print(f"✓ 聚类模型训练完成 (K={optimal_k})")
    
    # 为训练集添加特征
    print("计算训练集特征...")
    train_df = _add_features_batch(
        train_df, train_locations, train_locations_scaled, 
        knn_models, data_center, cluster_model, is_train=True
    )
    
    result = {
        'train_df': train_df,
        'scaler': scaler,
        'knn_models': knn_models,
        'data_center': data_center,
        'cluster_model': cluster_model,
        'train_locations_scaled': train_locations_scaled,
        'optimal_k': optimal_k
    }
    
    # 处理验证集
    if test_df is not None:
        print("处理验证集...")
        test_df = test_df.copy()
        if '价格' in test_df.columns and '建筑面积' in test_df.columns:
            test_df["单价"] = test_df["价格"] / test_df["建筑面积"]
        
        test_locations = test_df[['lon', 'lat']].values
        test_locations_scaled = scaler.transform(test_locations)
        
        test_df = _add_features_batch(
            test_df, test_locations, test_locations_scaled,
            knn_models, data_center, cluster_model, 
            is_train=False, train_df=train_df
        )
        result['test_df'] = test_df
    
    # 处理预测集
    if predict_data is not None:
        print("处理预测集...")
        predict_data = predict_data.copy()
        if '价格' in predict_data.columns and '建筑面积' in predict_data.columns:
            predict_data["单价"] = predict_data["价格"] / predict_data["建筑面积"]
        
        predict_locations = predict_data[['lon', 'lat']].values
        predict_locations_scaled = scaler.transform(predict_locations)
        
        predict_data = _add_features_batch(
            predict_data, predict_locations, predict_locations_scaled,
            knn_models, data_center, cluster_model,
            is_train=False, train_df=train_df
        )
        result['predict_data'] = predict_data
    
    print("特征工程完成！")
    
    # 检查新特征是否正确添加
    check_new_features(result['train_df'], "训练集")
    if 'test_df' in result:
        check_new_features(result['test_df'], "验证集")
    if 'predict_data' in result:
        check_new_features(result['predict_data'], "预测集")
    
    return result

def _add_features_batch(df, locations, locations_scaled, knn_models, 
                       data_center, cluster_model, is_train=True, train_df=None):
    """批量添加特征，优化内存使用"""
    
    reference_df = df if is_train else train_df
    batch_size = 3000  # 分批处理，避免内存溢出
    total_samples = len(df)
    
    # 初始化特征列表
    features = {
        '地理位置评分': [],
        '到数据中心距离': [],
        '密度_1km': []
    }
    
    # 为每个K值初始化KNN特征
    for k in knn_models.keys():
        features[f'knn_{k}_mean'] = []
        features[f'knn_{k}_std'] = []
    
    # 如果有单价信息，添加排名特征
    if '单价' in df.columns:
        features['邻域价格排名'] = []
    
    # 聚类特征
    if cluster_model:
        features['cluster_id'] = []
        features['cluster_avg_price'] = []
    
    print(f"分批处理 {total_samples} 个样本...")
    
    # 分批处理
    for start_idx in range(0, total_samples, batch_size):
        end_idx = min(start_idx + batch_size, total_samples)
        batch_locations = locations[start_idx:end_idx]
        batch_locations_scaled = locations_scaled[start_idx:end_idx]
        
        print(f"处理批次 {start_idx}-{end_idx}...")
        
        # 1. 基础地理位置评分
        if 5 in knn_models:
            distances, indices = knn_models[5].kneighbors(batch_locations_scaled)
            batch_scores = []
            for i, idx_list in enumerate(indices):
                neighbor_prices = reference_df.iloc[idx_list]["单价"].values
                batch_scores.append(np.mean(neighbor_prices))
            features['地理位置评分'].extend(batch_scores)
        
        # 2. 到数据中心距离
        center_distances = np.sqrt(np.sum((batch_locations - data_center) ** 2, axis=1))
        features['到数据中心距离'].extend(center_distances.tolist())
        
        # 3. 简化的密度特征（只计算1km）
        reference_locations = reference_df[['lon', 'lat']].values
        radius_deg = 1.0 / 111.0  # 1km转换为度
        
        batch_densities = []
        for loc in batch_locations:
            distances_to_ref = np.sqrt(np.sum((reference_locations - loc) ** 2, axis=1))
            density = np.sum(distances_to_ref <= radius_deg)
            batch_densities.append(density)
        features['密度_1km'].extend(batch_densities)
        
        # 4. KNN统计特征
        for k in knn_models.keys():
            distances, indices = knn_models[k].kneighbors(batch_locations_scaled)
            
            batch_means = []
            batch_stds = []
            
            for idx_list in indices:
                neighbor_prices = reference_df.iloc[idx_list]["单价"].values
                batch_means.append(np.mean(neighbor_prices))
                batch_stds.append(np.std(neighbor_prices))
            
            features[f'knn_{k}_mean'].extend(batch_means)
            features[f'knn_{k}_std'].extend(batch_stds)
        
        # 5. 邻域价格排名（如果有单价）
        if '单价' in df.columns and 10 in knn_models:
            distances, indices = knn_models[10].kneighbors(batch_locations_scaled)
            batch_rankings = []
            
            for i, idx_list in enumerate(indices):
                current_price = df.iloc[start_idx + i]["单价"]
                neighbor_prices = reference_df.iloc[idx_list]["单价"].values
                rank = np.sum(neighbor_prices < current_price) / len(neighbor_prices)
                batch_rankings.append(rank)
            
            features['邻域价格排名'].extend(batch_rankings)
        
        # 6. 聚类特征
        if cluster_model:
            batch_clusters = cluster_model.predict(batch_locations_scaled)
            features['cluster_id'].extend(batch_clusters.tolist())
            
            # 计算簇平均价格
            batch_cluster_prices = []
            train_clusters = cluster_model.predict(reference_df[['lon', 'lat']].values 
                                                if hasattr(reference_df, 'values') 
                                                else locations_scaled if is_train 
                                                else train_df[['lon', 'lat']].values)
            
            for cluster_id in batch_clusters:
                same_cluster_mask = train_clusters == cluster_id
                if same_cluster_mask.sum() > 0:
                    cluster_price = reference_df[same_cluster_mask]["单价"].mean()
                else:
                    cluster_price = reference_df["单价"].mean()
                batch_cluster_prices.append(cluster_price)
            
            features['cluster_avg_price'].extend(batch_cluster_prices)
    
    # 将特征添加到DataFrame
    print(f"添加 {len(features)} 个新特征到DataFrame...")
    for feature_name, feature_values in features.items():
        if len(feature_values) == len(df):
            df[feature_name] = feature_values
            print(f"✓ 已添加特征: {feature_name}")
        else:
            print(f"✗ 特征长度不匹配: {feature_name} (期望{len(df)}, 实际{len(feature_values)})")
    
    print(f"DataFrame现在有 {len(df.columns)} 列")
    return df

def check_new_features(df, dataset_name="数据集"):
    """检查新增的地理位置特征"""
    location_features = [col for col in df.columns if any(keyword in col for keyword in 
                        ['地理位置', 'knn_', '密度', '距离', 'cluster'])]
    
    print(f"\n{dataset_name}新增特征检查:")
    print(f"总列数: {len(df.columns)}")
    print(f"地理特征数: {len(location_features)}")
    print("地理特征列表:")
    for i, feature in enumerate(location_features, 1):
        sample_values = df[feature].head(3).values
        print(f"  {i}. {feature}: {sample_values}")
    
    return location_features

def print_feature_summary(train_df, test_df=None, predict_df=None):
    """打印特征摘要"""
    
    location_features = [col for col in train_df.columns if any(keyword in col for keyword in 
                        ['地理位置', 'knn_', '密度', '距离', 'cluster'])]
    
    print("=" * 50)
    print("地理位置特征工程摘要")
    print("=" * 50)
    print(f"训练集: {len(train_df)} 样本")
    print(f"新增地理特征: {len(location_features)} 个")
    print("特征列表:", location_features)
    
    if test_df is not None:
        print(f"验证集: {len(test_df)} 样本")
    if predict_df is not None:
        print(f"预测集: {len(predict_df)} 样本")
    
    # 关键特征统计
    key_features = ['地理位置评分', 'knn_10_mean', '到数据中心距离']
    print("\n关键特征统计:")
    for feature in key_features:
        if feature in train_df.columns:
            mean_val = train_df[feature].mean()
            std_val = train_df[feature].std()
            print(f"{feature}: 均值={mean_val:.3f}, 标准差={std_val:.3f}")

# 使用示例
if __name__ == "__main__":
    result = compute_enhanced_location_features_improved(
        train_df, test_df, predict_data,
        auto_select_clusters=True,
        early_stop=True,      # 启用早停
        patience=3,           # 耐心值
        max_k=20,            # 仍然设置最大K值作为硬限制
        plot_metrics=True)

开始改进版地理位置特征工程...
训练集样本数: 71458
标准化地理位置坐标...
数据中心坐标: [113.36843402  35.2449446 ]
构建KNN模型...
✓ KNN-5 模型训练完成
✓ KNN-10 模型训练完成
自动选择最优聚类数...
寻找最优K值，范围: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
早停设置: patience=3, min_improvement=0.02
K=2: WSS=44281.32, Silhouette=0.661
K=3: WSS=21685.15, Silhouette=0.726
  ✓ 发现改善: +1.7262
K=4: WSS=9711.92, Silhouette=0.792
  ✓ 发现改善: +0.0655
K=5: WSS=2680.00, Silhouette=0.921
  ✓ 发现改善: +0.1295
K=6: WSS=63.74, Silhouette=0.972
  ✓ 发现改善: +0.0505
K=7: WSS=49.14, Silhouette=0.839
  → 无显著改善 (1/3)
K=8: WSS=42.70, Silhouette=0.848
  → 无显著改善 (2/3)
K=9: WSS=38.17, Silhouette=0.763
  → 无显著改善 (3/3)
🛑 早停触发: 连续3次无改善，当前最佳K=6
  但WSS仍在快速下降(0.106), 继续搜索...
K=10: WSS=34.66, Silhouette=0.647
  → 无显著改善 (3/3)
🛑 早停触发: 连续3次无改善，当前最佳K=6

📊 结果分析:
肘部法则推荐K: 5
轮廓系数最佳K: 6
早停建议K: 6
最终选择K: 6 (多方法一致)


<Figure size 864x288 with 2 Axes>

✓ 聚类模型训练完成 (K=6)
计算训练集特征...
分批处理 71458 个样本...
处理批次 0-3000...
处理批次 3000-6000...
处理批次 6000-9000...
处理批次 9000-12000...
处理批次 12000-15000...
处理批次 15000-18000...
处理批次 18000-21000...
处理批次 21000-24000...
处理批次 24000-27000...
处理批次 27000-30000...
处理批次 30000-33000...
处理批次 33000-36000...
处理批次 36000-39000...
处理批次 39000-42000...
处理批次 42000-45000...
处理批次 45000-48000...
处理批次 48000-51000...
处理批次 51000-54000...
处理批次 54000-57000...
处理批次 57000-60000...
处理批次 60000-63000...
处理批次 63000-66000...
处理批次 66000-69000...
处理批次 69000-71458...
添加 10 个新特征到DataFrame...
✓ 已添加特征: 地理位置评分
✓ 已添加特征: 到数据中心距离
✓ 已添加特征: 密度_1km
✓ 已添加特征: knn_5_mean
✓ 已添加特征: knn_5_std
✓ 已添加特征: knn_10_mean
✓ 已添加特征: knn_10_std
✓ 已添加特征: 邻域价格排名
✓ 已添加特征: cluster_id
✓ 已添加特征: cluster_avg_price
DataFrame现在有 195 列
处理验证集...
分批处理 12619 个样本...
处理批次 0-3000...
处理批次 3000-6000...
处理批次 6000-9000...
处理批次 9000-12000...
处理批次 12000-12619...
添加 10 个新特征到DataFrame...
✓ 已添加特征: 地理位置评分
✓ 已添加特征: 到数据中心距离
✓ 已添加特征: 密度_1km
✓ 已添加特征: knn_5_mean
✓ 已添加特征: knn_5_std
✓ 已添加特征: knn_10_mean

In [46]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

def compute_enhanced_location_features(train_df, test_df=None, predict_data=None, 
                                     min_k=5, max_k=15, lightweight_mode=True):
    """
    轻量级增强版地理位置特征工程
    
    参数:
    - train_df: 训练集DataFrame
    - test_df: 验证集DataFrame（可选）
    - predict_data: 预测集DataFrame（可选）
    - min_k, max_k: KNN参数
    - lightweight_mode: 轻量级模式，减少内存使用
    """
    
    print("开始地理位置特征工程...")
    
    # 检查必要列
    required_columns = ['价格', '建筑面积', 'lon', 'lat']
    for col in required_columns:
        if col not in train_df.columns:
            raise ValueError(f"训练集中未找到 '{col}' 列")
    
    # 处理训练集
    train_df = train_df.copy()
    train_df["单价"] = train_df["价格"] / train_df["建筑面积"]
    train_locations = train_df[['lon', 'lat']].values
    
    print(f"训练集样本数: {len(train_df)}")
    
    # 标准化和KNN
    scaler = StandardScaler()
    train_locations_scaled = scaler.fit_transform(train_locations)
    
    # 只使用必要的K值，减少内存占用
    k_values = [5, 10] if lightweight_mode else [5, 10, 20]
    knn_models = {}
    
    for k in k_values:
        actual_k = min(k, len(train_df) - 1)
        if actual_k > 0:
            print(f"训练KNN模型 K={actual_k}...")
            knn_models[k] = NearestNeighbors(n_neighbors=actual_k, algorithm='ball_tree')
            knn_models[k].fit(train_locations_scaled)
    
    # 计算数据中心
    data_center = np.mean(train_locations, axis=0)
    
    # 简化聚类（只用一个小的K-means）
    cluster_model = None
    if len(train_df) > 20 and not lightweight_mode:
        try:
            print("训练聚类模型...")
            n_clusters = min(10, len(train_df) // 5)  # 动态调整簇数
            cluster_model = KMeans(n_clusters=n_clusters, random_state=42, n_init=5)
            cluster_model.fit(train_locations_scaled)
        except Exception as e:
            print(f"聚类训练失败，跳过: {e}")
    
    # 为训练集添加特征
    print("计算训练集特征...")
    train_df = _add_features_batch(
        train_df, train_locations, train_locations_scaled, 
        knn_models, data_center, cluster_model, is_train=True
    )
    
    result = {
        'train_df': train_df,
        'scaler': scaler,
        'knn_models': knn_models,
        'data_center': data_center,
        'cluster_model': cluster_model,
        'train_locations_scaled': train_locations_scaled
    }
    
    # 处理验证集
    if test_df is not None:
        print("处理验证集...")
        test_df = test_df.copy()
        if '价格' in test_df.columns and '建筑面积' in test_df.columns:
            test_df["单价"] = test_df["价格"] / test_df["建筑面积"]
        
        test_locations = test_df[['lon', 'lat']].values
        test_locations_scaled = scaler.transform(test_locations)
        
        test_df = _add_features_batch(
            test_df, test_locations, test_locations_scaled,
            knn_models, data_center, cluster_model, 
            is_train=False, train_df=train_df
        )
        result['test_df'] = test_df
    
    # 处理预测集
    if predict_data is not None:
        print("处理预测集...")
        predict_data = predict_data.copy()
        if '价格' in predict_data.columns and '建筑面积' in predict_data.columns:
            predict_data["单价"] = predict_data["价格"] / predict_data["建筑面积"]
        
        predict_locations = predict_data[['lon', 'lat']].values
        predict_locations_scaled = scaler.transform(predict_locations)
        
        predict_data = _add_features_batch(
            predict_data, predict_locations, predict_locations_scaled,
            knn_models, data_center, cluster_model,
            is_train=False, train_df=train_df
        )
        result['predict_data'] = predict_data
    
    print("特征工程完成！")
    
    # 检查新特征是否正确添加
    check_new_features(result['train_df'], "训练集")
    if 'test_df' in result:
        check_new_features(result['test_df'], "验证集")
    if 'predict_data' in result:
        check_new_features(result['predict_data'], "预测集")
    
    return result

def _add_features_batch(df, locations, locations_scaled, knn_models, 
                       data_center, cluster_model, is_train=True, train_df=None):
    """批量添加特征，优化内存使用"""
    
    reference_df = df if is_train else train_df
    batch_size = 3000  # 分批处理，避免内存溢出
    total_samples = len(df)
    
    # 初始化特征列表
    features = {
        '地理位置评分': [],
        '到数据中心距离': [],
        '密度_1km': []
    }
    
    # 为每个K值初始化KNN特征
    for k in knn_models.keys():
        features[f'knn_{k}_mean'] = []
        features[f'knn_{k}_std'] = []
    
    # 如果有单价信息，添加排名特征
    if '单价' in df.columns:
        features['邻域价格排名'] = []
    
    # 聚类特征
    if cluster_model:
        features['cluster_id'] = []
        features['cluster_avg_price'] = []
    
    print(f"分批处理 {total_samples} 个样本...")
    
    # 分批处理
    for start_idx in range(0, total_samples, batch_size):
        end_idx = min(start_idx + batch_size, total_samples)
        batch_locations = locations[start_idx:end_idx]
        batch_locations_scaled = locations_scaled[start_idx:end_idx]
        
        print(f"处理批次 {start_idx}-{end_idx}...")
        
        # 1. 基础地理位置评分
        if 5 in knn_models:
            distances, indices = knn_models[5].kneighbors(batch_locations_scaled)
            batch_scores = []
            for i, idx_list in enumerate(indices):
                neighbor_prices = reference_df.iloc[idx_list]["单价"].values
                batch_scores.append(np.mean(neighbor_prices))
            features['地理位置评分'].extend(batch_scores)
        
        # 2. 到数据中心距离
        center_distances = np.sqrt(np.sum((batch_locations - data_center) ** 2, axis=1))
        features['到数据中心距离'].extend(center_distances.tolist())
        
        # 3. 简化的密度特征（只计算1km）
        reference_locations = reference_df[['lon', 'lat']].values
        radius_deg = 1.0 / 111.0  # 1km转换为度
        
        batch_densities = []
        for loc in batch_locations:
            distances_to_ref = np.sqrt(np.sum((reference_locations - loc) ** 2, axis=1))
            density = np.sum(distances_to_ref <= radius_deg)
            batch_densities.append(density)
        features['密度_1km'].extend(batch_densities)
        
        # 4. KNN统计特征
        for k in knn_models.keys():
            distances, indices = knn_models[k].kneighbors(batch_locations_scaled)
            
            batch_means = []
            batch_stds = []
            
            for idx_list in indices:
                neighbor_prices = reference_df.iloc[idx_list]["单价"].values
                batch_means.append(np.mean(neighbor_prices))
                batch_stds.append(np.std(neighbor_prices))
            
            features[f'knn_{k}_mean'].extend(batch_means)
            features[f'knn_{k}_std'].extend(batch_stds)
        
        # 5. 价格排名特征
        if '单价' in df.columns and 10 in knn_models:
            distances, indices = knn_models[10].kneighbors(batch_locations_scaled)
            batch_rankings = []
            
            for i, idx_list in enumerate(indices):
                neighbor_prices = reference_df.iloc[idx_list]["单价"].values
                current_price = df.iloc[start_idx + i]["单价"]
                rank = (neighbor_prices < current_price).sum() / len(neighbor_prices)
                batch_rankings.append(rank)
            
            features['邻域价格排名'].extend(batch_rankings)
        
        # 6. 聚类特征
        if cluster_model:
            batch_clusters = cluster_model.predict(batch_locations_scaled)
            features['cluster_id'].extend(batch_clusters.tolist())
            
            # 计算簇平均价格
            batch_cluster_prices = []
            for cluster_id in batch_clusters:
                train_clusters = cluster_model.predict(locations_scaled if is_train else 
                                                    cluster_model.cluster_centers_)
                same_cluster_mask = train_clusters == cluster_id
                if same_cluster_mask.sum() > 0:
                    cluster_price = reference_df[same_cluster_mask]["单价"].mean()
                else:
                    cluster_price = reference_df["单价"].mean()
                batch_cluster_prices.append(cluster_price)
            
            features['cluster_avg_price'].extend(batch_cluster_prices)
    
    # 将特征添加到DataFrame
    print(f"添加 {len(features)} 个新特征到DataFrame...")
    for feature_name, feature_values in features.items():
        if len(feature_values) == len(df):
            df[feature_name] = feature_values
            print(f"✓ 已添加特征: {feature_name}")
        else:
            print(f"✗ 特征长度不匹配: {feature_name} (期望{len(df)}, 实际{len(feature_values)})")
    
    print(f"DataFrame现在有 {len(df.columns)} 列")
    return df

def check_new_features(df, dataset_name="数据集"):
    """检查新增的地理位置特征"""
    location_features = [col for col in df.columns if any(keyword in col for keyword in 
                        ['地理位置', 'knn_', '密度', '距离', '排名', 'cluster'])]
    
    print(f"\n{dataset_name}新增特征检查:")
    print(f"总列数: {len(df.columns)}")
    print(f"地理特征数: {len(location_features)}")
    print("地理特征列表:")
    for i, feature in enumerate(location_features, 1):
        sample_values = df[feature].head(3).values
        print(f"  {i}. {feature}: {sample_values}")
    
    return location_features

def print_feature_summary(train_df, test_df=None, predict_df=None):
    """打印特征摘要"""
    
    location_features = [col for col in train_df.columns if any(keyword in col for keyword in 
                        ['地理位置', 'knn_', '密度', '距离', '排名', 'cluster'])]
    
    print("=" * 50)
    print("地理位置特征工程摘要")
    print("=" * 50)
    print(f"训练集: {len(train_df)} 样本")
    print(f"新增地理特征: {len(location_features)} 个")
    print("特征列表:", location_features)
    
    if test_df is not None:
        print(f"验证集: {len(test_df)} 样本")
    if predict_df is not None:
        print(f"预测集: {len(predict_df)} 样本")
    
    # 关键特征统计
    key_features = ['地理位置评分', 'knn_10_mean', '到数据中心距离']
    print("\n关键特征统计:")
    for feature in key_features:
        if feature in train_df.columns:
            mean_val = train_df[feature].mean()
            std_val = train_df[feature].std()
            print(f"{feature}: 均值={mean_val:.3f}, 标准差={std_val:.3f}")

# 使用示例
if __name__ == "__main__":
    print("轻量级地理位置特征工程已准备就绪！")
    print("\n使用方法:")
    print("# 轻量级模式（推荐）")
    result = compute_enhanced_location_features(train_df, test_df, predict_data, lightweight_mode=True)
    print("\n# 检查新特征")
    enhanced_train = result['train_df']
    print('新增特征:', [col for col in enhanced_train.columns if 'knn_' in col or '地理位置' in col])
    print('训练集形状:', enhanced_train.shape)
    print("\n# 查看结果摘要")
    print_feature_summary(result['train_df'], result.get('test_df'))
    
    print("\n🔍 特征检查方法:")
    print("# 直接查看DataFrame的列")
    print(enhanced_train.columns.tolist())
    print("# 查看特定特征的值")
    print(enhanced_train[['地理位置评分', 'knn_5_mean', '到数据中心距离']].head())

轻量级地理位置特征工程已准备就绪！

使用方法:
# 轻量级模式（推荐）
开始地理位置特征工程...
训练集样本数: 71458
训练KNN模型 K=5...
训练KNN模型 K=10...
计算训练集特征...
分批处理 71458 个样本...
处理批次 0-3000...
处理批次 3000-6000...
处理批次 6000-9000...
处理批次 9000-12000...
处理批次 12000-15000...
处理批次 15000-18000...
处理批次 18000-21000...
处理批次 21000-24000...
处理批次 24000-27000...
处理批次 27000-30000...
处理批次 30000-33000...
处理批次 33000-36000...
处理批次 36000-39000...
处理批次 39000-42000...
处理批次 42000-45000...
处理批次 45000-48000...
处理批次 48000-51000...
处理批次 51000-54000...
处理批次 54000-57000...
处理批次 57000-60000...
处理批次 60000-63000...
处理批次 63000-66000...
处理批次 66000-69000...
处理批次 69000-71458...
添加 8 个新特征到DataFrame...
✓ 已添加特征: 地理位置评分
✓ 已添加特征: 到数据中心距离
✓ 已添加特征: 密度_1km
✓ 已添加特征: knn_5_mean
✓ 已添加特征: knn_5_std
✓ 已添加特征: knn_10_mean
✓ 已添加特征: knn_10_std
✓ 已添加特征: 邻域价格排名
DataFrame现在有 173 列
处理验证集...
分批处理 12619 个样本...
处理批次 0-3000...
处理批次 3000-6000...
处理批次 6000-9000...
处理批次 9000-12000...
处理批次 12000-12619...
添加 8 个新特征到DataFrame...
✓ 已添加特征: 地理位置评分
✓ 已添加特征: 到数据中心距离
✓ 已添加特征: 密度_1km
✓ 已添加特征: knn_5_mean
✓ 已添加特征: 

In [49]:
enhanced_train = result['train_df']
enhanced_test=result['test_df']
enhanced_predict=result['predict_data']

In [51]:
# 检查数据集中的列
print("当前所有列名:", enhanced_predict.columns.tolist())

当前所有列名: ['城市', '区域', '板块', '环线', '小区名称', '房屋户型', '所在楼层', '建筑面积', '梯户比例', '配备电梯', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', 'ID', '匹配小区', '建筑年代', '房屋总数', '楼栋总数', '绿 化 率', '容 积 率', '物 业 费', '停车位', '停车费用', '平均每平米月租金', '交易年份', '建筑结构编码_混合结构', '建筑结构编码_钢混结构', '装修编码_毛坯', '装修编码_简装', '装修编码_精装', '房屋数量', '楼栋数量', '绿化率', '容积率', '物业费', '停车位数量', '停车费', 'poly_房屋数量', 'poly_楼栋数量', 'poly_绿化率', 'poly_容积率', 'poly_物业费', 'poly_停车位数量', 'poly_停车费', 'poly_房屋数量^2', 'poly_房屋数量 楼栋数量', 'poly_房屋数量 绿化率', 'poly_房屋数量 容积率', 'poly_房屋数量 物业费', 'poly_房屋数量 停车位数量', 'poly_房屋数量 停车费', 'poly_楼栋数量^2', 'poly_楼栋数量 绿化率', 'poly_楼栋数量 容积率', 'poly_楼栋数量 物业费', 'poly_楼栋数量 停车位数量', 'poly_楼栋数量 停车费', 'poly_绿化率^2', 'poly_绿化率 容积率', 'poly_绿化率 物业费', 'poly_绿化率 停车位数量', 'poly_绿化率 停车费', 'poly_容积率^2', 'poly_容积率 物业费', 'poly_容积率 停车位数量', 'poly_容积率 停车费', 'poly_物业费^2', 'poly_物业费 停车位数量', 'poly_物业费 停车费', 'poly_停车位数量^2', 'poly_停车位数量 停车费', 'poly_停车费^2', 'area_scaled', 'area_squared', '主要朝向_北', '主要朝向_南', '主要朝向_西', '环线_填补', '城市_环线_0_二环内', '城市_环线_0_二至三环'

# 训练模型

In [53]:
# 检查 DataFrame 是否包含稀疏列
if any(pd.api.types.is_sparse(enhanced_train[col]) for col in enhanced_train.columns):
    print("DataFrame 包含稀疏列！")
else:
    print("DataFrame 不含稀疏列，可能是普通数值数据。")

# 如果数据是 pandas.DataFrame 且含稀疏列
for col in enhanced_train.columns:
    if pd.api.types.is_sparse(enhanced_train[col]):
       enhanced_train[col] = enhanced_train[col].sparse.to_dense()  # 逐列转换
for col in enhanced_test.columns:
    if pd.api.types.is_sparse(enhanced_test[col]):
       enhanced_test[col] = enhanced_test[col].sparse.to_dense()
for col in enhanced_predict.columns:
    if pd.api.types.is_sparse(enhanced_predict[col]):
         enhanced_predict[col] = enhanced_predict[col].sparse.to_dense()

DataFrame 包含稀疏列！


In [56]:
selected_features =['区域','板块','area_scaled', 'area_squared','配备电梯', '平均每平米月租金',
               '室', '厅', '厨', '卫', '房龄_平方','户梯比',
               '房龄', '楼层占比','面积_房龄','楼层_电梯交互', '楼层_高层交互','交通_得分', '配套_得分', 
               '学区_得分', '税费_得分', '描述丰富度', '房屋优势_长度', '核心卖点_长度', '周边配套_长度', 
               '交通出行_长度', '综合文本得分', '房屋优势_长度_log', '核心卖点_长度_log', '周边配套_长度_log', 
               '交通出行_长度_log', '综合文本得分_scaled', '描述丰富度_分档', '核心卖点_特征_encoded', '学区交通_交互', '配套描述_交互',
               '建筑结构编码_混合结构', '建筑结构编码_钢混结构', '装修编码_简装', '装修编码_精装','装修编码_毛坯',
                *city_ring_columns,*[f"{col}_建筑面积" for col in city_ring_columns],
               '绿化率', '容积率', '物业费', '停车位数量', '停车费', 'poly_房屋数量', 'poly_楼栋数量', 'poly_绿化率', 
               'poly_容积率', 'poly_物业费', 'poly_停车位数量', 'poly_停车费', 'poly_房屋数量^2', 'poly_房屋数量 楼栋数量', 
               'poly_房屋数量 绿化率', 'poly_房屋数量 容积率', 'poly_房屋数量 物业费', 'poly_房屋数量 停车位数量', 'poly_房屋数量 停车费', 
               'poly_楼栋数量^2', 'poly_楼栋数量 绿化率', 'poly_楼栋数量 容积率', 'poly_楼栋数量 物业费', 'poly_楼栋数量 停车位数量', 
               'poly_楼栋数量 停车费', 'poly_绿化率^2', 'poly_绿化率 容积率', 'poly_绿化率 物业费', 'poly_绿化率 停车位数量',
               'poly_绿化率 停车费', 'poly_容积率^2', 'poly_容积率 物业费', 'poly_容积率 停车位数量', 'poly_容积率 停车费', 
               'poly_物业费^2', 'poly_物业费 停车位数量', 'poly_物业费 停车费', 'poly_停车位数量^2', 'poly_停车位数量 停车费', 'poly_停车费^2',
               '主要朝向_北', '主要朝向_南', '主要朝向_西','单位租金_平方', '卧室_租金', '面积_租金','地理位置评分', '到数据中心距离', '密度_1km',
               'knn_5_mean', 'knn_5_std', 'knn_10_mean', 'knn_10_std', 'cluster_id', 'cluster_avg_price']

# 因变量
y_train = enhanced_train['log_价格']
y_test = enhanced_test['log_价格']

X_train = enhanced_train[selected_features]
X_test = enhanced_test[selected_features]

#新增
X_train = pd.DataFrame(X_train, columns=selected_features) 
X_test = pd.DataFrame(X_test, columns=selected_features)

# 定义评估函数同时计算MAE和RMSE
def evaluate_model(model, X_train, y_train, X_test, y_test):
    # 训练模型
    model.fit(X_train, y_train)
    
    # 预测
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # 计算原始价格的评估指标 (将log转换回原始价格)
    y_train_original = np.exp(y_train)
    y_train_pred_original = np.exp(y_train_pred)
    y_test_original = np.exp(y_test)
    y_test_pred_original = np.exp(y_test_pred)

    print("是否存在 NaN/Inf 或值爆炸？")
    print("y_train_original 中 NaN：", np.any(np.isnan(y_train_original)))
    print("y_train_pred_original 中 NaN：", np.any(np.isnan(y_train_pred_original)))
    print("y_train_pred_original 最大值：", np.max(y_train_pred_original))
    print("y_train_pred 原始最大 log 值：", np.max(y_train_pred))

    
    # 训练集指标
    train_mae = mean_absolute_error(y_train_original, y_train_pred_original)
    train_rmse = np.sqrt(mean_squared_error(y_train_original, y_train_pred_original))
    
    # 测试集指标
    test_mae = mean_absolute_error(y_test_original, y_test_pred_original)
    test_rmse = np.sqrt(mean_squared_error(y_test_original, y_test_pred_original))

    # 输出评估指标
    print(f"Train MAE: {train_mae:.2f}, Train RMSE: {train_rmse:.2f}")
    print(f"Test MAE: {test_mae:.2f}, Test RMSE: {test_rmse:.2f}")

    return {
        "train_mae": train_mae,
        "train_rmse": train_rmse,
        "test_mae": test_mae,
        "test_rmse": test_rmse
    }

In [57]:
# XGboost 基础版本
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

xgb_model = xgb.XGBRegressor(
    n_estimators=1000,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# 使用你现有的评估函数
results = evaluate_model(xgb_model, X_train, y_train, X_test, y_test)

是否存在 NaN/Inf 或值爆炸？
y_train_original 中 NaN： False
y_train_pred_original 中 NaN： False
y_train_pred_original 最大值： 79317600.0
y_train_pred 原始最大 log 值： 18.18897
Train MAE: 83911.24, Train RMSE: 164810.96
Test MAE: 157161.25, Test RMSE: 519124.44


In [58]:
# 假设你已经 fit 过模型
#xgb_model.fit(X_train, y_train)

# 取特征名
feature_names = X_train.columns

# 获得特征重要性分数
importances = xgb_model.feature_importances_

# 转为DataFrame，排序查看
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print(importance_df.head(20))  # 显示前20个

          Feature  Importance
148   knn_10_mean    0.466871
143        地理位置评分    0.288315
146    knn_5_mean    0.118367
2     area_scaled    0.017307
14          面积_房龄    0.015845
142         面积_租金    0.013584
141         卧室_租金    0.013492
13           楼层占比    0.003557
9               卫    0.003492
1              板块    0.003275
15        楼层_电梯交互    0.001698
66    城市_环线_6_三环外    0.001575
3    area_squared    0.001446
45   城市_环线_0_四至五环    0.001361
6               室    0.001198
150    cluster_id    0.001011
55    城市_环线_3_外环外    0.000943
60   城市_环线_4_二至三环    0.000882
149    knn_10_std    0.000873
147     knn_5_std    0.000825


In [63]:
# 1. 定义优化后的模型
optimized_xgbmodel = xgb.XGBRegressor(
    n_estimators=710,
    max_depth=7,
    subsample=0.772,
    colsample_bytree=0.858,
    learning_rate=0.082,
    random_state=42
)

# 2. 训练并评估
optimized_results = evaluate_model(optimized_xgbmodel, X_train, y_train, X_test, y_test)

# 3. 对比基础版
print("基础版结果:", results)
print("优化版结果:", optimized_results)

是否存在 NaN/Inf 或值爆炸？
y_train_original 中 NaN： False
y_train_pred_original 中 NaN： False
y_train_pred_original 最大值： 77940130.0
y_train_pred 原始最大 log 值： 18.171452
Train MAE: 103785.58, Train RMSE: 213961.37
Test MAE: 160714.72, Test RMSE: 553815.19
基础版结果: {'train_mae': 83911.2417835258, 'train_rmse': 164810.9647245222, 'test_mae': 157161.2453783729, 'test_rmse': 519124.4441576613}
优化版结果: {'train_mae': 103785.58096965612, 'train_rmse': 213961.3724145273, 'test_mae': 160714.72009531755, 'test_rmse': 553815.188282513}


In [65]:
import numpy as np
import pandas as pd

# 1. 选取特征
id_col = enhanced_predict['ID'].copy()
predict_features = enhanced_predict.drop(columns=['ID'])
predict_features=enhanced_predict[selected_features]
# 2. 使用训练好的模型进行预测（模型预测的是 log_价格）
pred_log_price = optimized_xgbmodel.predict(predict_features)

# 3. 将预测的 log_价格 还原为原始价格
pred_price = np.exp(pred_log_price)

# 4. 将预测价格转换为整数
pred_price_int = np.round(pred_price).astype(int)

# 5. 构造结果 DataFrame
submission = pd.DataFrame({
    'ID': id_col,
    'Price': pred_price_int
})

# 6. 保存为 CSV 文件
submission.to_csv("prediction_xgboost_K3.csv", index=False)
print("预测结果已保存到 prediction_xgboost_K3.csv")

预测结果已保存到 prediction_xgboost_K3.csv


In [59]:
# XGBoost调参（Optuna）
import optuna
from sklearn.metrics import mean_squared_error

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 500, 1200),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist"  # 推荐加速训练
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)

    # 预测（还原价格空间）
    y_pred_log = model.predict(X_test)
    y_pred = np.exp(y_pred_log)
    y_true = np.exp(y_test)

    # 计算 RMSE（原始价格空间）
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse  # optuna 默认 minimize

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=60)

[I 2025-06-01 15:06:08,290] A new study created in memory with name: no-name-813c9c0b-2893-4abd-aa2e-77e0948a242e
[I 2025-06-01 15:06:27,662] Trial 0 finished with value: 594691.652351822 and parameters: {'n_estimators': 781, 'max_depth': 7, 'learning_rate': 0.015350339082741374, 'subsample': 0.8613787076851702, 'colsample_bytree': 0.9133879530269682, 'min_child_weight': 10, 'gamma': 0.1662562108588983}. Best is trial 0 with value: 594691.652351822.
[I 2025-06-01 15:06:38,361] Trial 1 finished with value: 624404.7074830135 and parameters: {'n_estimators': 778, 'max_depth': 4, 'learning_rate': 0.03229064764655059, 'subsample': 0.8812536071850743, 'colsample_bytree': 0.8333827947386697, 'min_child_weight': 10, 'gamma': 0.04283411824139938}. Best is trial 0 with value: 594691.652351822.
[I 2025-06-01 15:06:49,497] Trial 2 finished with value: 749818.3262821162 and parameters: {'n_estimators': 1003, 'max_depth': 7, 'learning_rate': 0.03625358481003018, 'subsample': 0.7017421226391961, 'col

In [61]:
from sklearn.model_selection import KFold

def fast_optuna_objective_cv(trial, X_train, y_train, n_splits=3):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'random_state': 42,
        'n_estimators': trial.suggest_int('n_estimators', 600, 1200),
        'max_depth': trial.suggest_int('max_depth', 5, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.8, 1.0),
    }
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=trial.number)
    cv_scores = []
    for train_idx, val_idx in cv.split(X_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        model = xgb.XGBRegressor(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], early_stopping_rounds=15, verbose=False)
        pred = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, pred))
        cv_scores.append(rmse)
    return np.mean(cv_scores)

def run_fast_optuna_cv(X_train, y_train, n_trials=30, n_splits=3):
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(lambda trial: fast_optuna_objective_cv(trial, X_train, y_train, n_splits), n_trials=n_trials, show_progress_bar=True)
    return study.best_params, study

# 用法
best_params, study = run_fast_optuna_cv(X_train, y_train, n_trials=30, n_splits=3)
model = xgb.XGBRegressor(**best_params)
model.fit(X_train, y_train)

# 训练完成后再用 X_test, y_test 做最终评估
from sklearn.metrics import mean_squared_error, mean_absolute_error
y_test_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
mae = mean_absolute_error(y_test, y_test_pred)
print(f"Test RMSE: {rmse:.4f}, MAE: {mae:.4f}")

[I 2025-06-01 15:23:49,828] A new study created in memory with name: no-name-2dda68ba-2c7f-45e5-8a93-d2fcf736684e


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-06-01 15:25:36,782] Trial 0 finished with value: 0.12229841235231476 and parameters: {'n_estimators': 825, 'max_depth': 12, 'learning_rate': 0.14518497439122724, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.8312037280884873}. Best is trial 0 with value: 0.12229841235231476.
[I 2025-06-01 15:27:58,960] Trial 1 finished with value: 0.12360377383211163 and parameters: {'n_estimators': 693, 'max_depth': 5, 'learning_rate': 0.20880081501089318, 'subsample': 0.8404460046972835, 'colsample_bytree': 0.941614515559209}. Best is trial 0 with value: 0.12229841235231476.
[I 2025-06-01 15:29:05,508] Trial 2 finished with value: 0.12584550845520026 and parameters: {'n_estimators': 612, 'max_depth': 12, 'learning_rate': 0.19057174434080473, 'subsample': 0.6849356442713105, 'colsample_bytree': 0.8363649934414201}. Best is trial 0 with value: 0.12229841235231476.
[I 2025-06-01 15:33:19,568] Trial 3 finished with value: 0.11975548889655181 and parameters: {'n_estimators': 710, 'max_dep

KeyboardInterrupt: 

In [62]:
print("Best Trial:")
print(study.best_trial.params)
print(f"Best RMSE: {study.best_value:.2f}")

# 用最优参数重新训练模型
best_params = study.best_trial.params
final_model = xgb.XGBRegressor(**best_params)
final_model.fit(X_train, y_train)

# 使用你的 evaluate_model 函数评估
evaluate_model(final_model, X_train, y_train, X_test, y_test)

Best Trial:
{'n_estimators': 942, 'max_depth': 10, 'learning_rate': 0.03959063648664691, 'subsample': 0.9356031520739537, 'colsample_bytree': 0.8959187926548916, 'min_child_weight': 9, 'gamma': 0.0028728519722198776}
Best RMSE: 528626.70


In [64]:
import numpy as np
import pandas as pd

# 1. 选取特征
id_col = processed_predict['ID'].copy()
predict_features = processed_predict.drop(columns=['ID'])
predict_features=processed_predict[selected_features]
# 2. 使用训练好的模型进行预测（模型预测的是 log_价格）
pred_log_price = final_model.predict(predict_features)

# 3. 将预测的 log_价格 还原为原始价格
pred_price = np.exp(pred_log_price)

# 4. 将预测价格转换为整数
pred_price_int = np.round(pred_price).astype(int)

# 5. 构造结果 DataFrame
submission = pd.DataFrame({
    'ID': id_col,
    'Price': pred_price_int
})

# 6. 保存为 CSV 文件
submission.to_csv("prediction_xgboost_Koptuna.csv", index=False)
print("预测结果已保存到 prediction_xgboost_Koptuna.csv")